# Importação das Bibliotecas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, MeanShift, estimate_bandwidth
from sklearn.model_selection import ParameterGrid, ParameterSampler
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from scipy.cluster.hierarchy import dendrogram, linkage
from mpl_toolkits.mplot3d import Axes3D
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV

# from hdbscan import HDBSCAN

1. Neste bloco realizamos a importação das bibliotecas utilizadas para análise dos dados e treinamento do modelo

# Carregar a Base de Dados

In [2]:
df = pd.read_csv('../documents/extras/base_dados.csv')

1. Nesta etapa carregamos a base de dados da Unipar para efetuarmos todas as análises.
    - Segue o _link_ da base de dados para facilitar a correção **(Só é possível ter acesso a este arquivo caso você esteja autenticado com uma conta com o e-mail institucional do Inteli)**: [Base de Dados](https://drive.google.com/file/d/18xMCvMC2UTvYebN-ZXvJw_0eZM8WCu4y/view?usp=sharing)
    - Posicione o arquivo base_dados.csv na pasta **/documents/extras/**

# Identificação das colunas

In [ ]:
df.info()

1. Nesta Etapa tipificamos as colunas como numéricas ou categóricas para prosseguirmos com o tratamento dos dados.

| **Nº** | **Coluna**                     | **Tipo**       |
|--------|---------------------------------|----------------|
| 1      | Apolice Sinistro                | Categórica     |
| 2      | Codigo Empresa Sinistro         | Categórica       |
| 3      | Nome Empresa Sinistro           | Categórica     |
| 4      | SEGURADO                        | Categórica       |
| 5      | Codigo Especialidade Sinistro   | Categórica       |
| 6      | Elegibilidade Sinistro          | Categórica     |
| 7      | Sexo Sinistro                   | Categórica     |
| 8      | Faixa-Etária Nova Sinistro      | Categórica     |
| 9      | Descricao Plano Sinistro        | Categórica     |
| 10     | Codigo Servico Sinistro         | Categórica       |
| 11     | Descricao Servico Sinistro      | Categórica     |
| 12     | Tipo Utilização Sinistro        | Categórica     |
| 13     | Dt Data Sinistro                | Categórica     |
| 14     | Codigo Prestador                | Categórica       |
| 15     | Nome Prestador Sinistro         | Categórica     |
| 16     | Valor Pago Sinistro             | Numérica     |
| 17     | Valor Usuario Sinistro          | Numérica       |
| 18     | Quantidade                      | Numérica       |
| 19     | No Ano Mes                      | Categórica       |
| 20     | Codigo Grupo Empresa            | Categórica       |
| 21     | Nome Grupo Empresa              | Categórica     |


# Análise Descritiva das Colunas

In [ ]:
df.describe()

1. Após realizarmos esta análise, podemos observar que a grande maioria das colunas númericas são colunas para identificação. 

# Análise Descritiva das Colunas Selecionadas

In [ ]:
print('Quantidade:')
print(df['Quantidade'].describe())

df['Valor Pago Sinistro'] = df['Valor Pago Sinistro'].replace(",", ".", regex=True).astype(float)

print("Valor Pago Sinistro:")
print(f"min: {df['Valor Pago Sinistro'].min()}")
print(f"max: {df['Valor Pago Sinistro'].max()}")
print(f"média: {df['Valor Pago Sinistro'].mean()}")
print(f"mediana: {df['Valor Pago Sinistro'].median()}")
print(f"desvio padrão: {df['Valor Pago Sinistro'].std()}")

print('Valor Usuario Sinistro:')
print(df['Valor Usuario Sinistro'].describe())


1. Inicialmente visualizamos as estatísticas descritivas da coluna Quantidade.
2. Formatação da coluna Valor Pago Sinistro, pois para realizarmos a análise foi preciso alterar a formatação dos valores que estavam sendo separados por vírgula, alterando para ponto.
3. Visualização das estatísticas descritivas da coluna Valor Pago Sinistro.
4. Visualização das estatísticas descritivas da coluna Valor Usuario Sinistro, após essa análise podemos observar que essa coluna contém apenas o valor 0 em todas as linas, a tornando uma coluna inutilizável para o treinamento do modelo preditivo.

# Gráficos


In [6]:
colunas_titular = df[df["Elegibilidade Sinistro"] == "TITULAR"]

1. Nesta primeira etapa criamos uma variável que filtra a base de dados para utilizar apenas linhas que tenhas "TITULAR" na coluna Elegibilidade Sinistro. Dessa forma, as análises serão direcionadas para serviços que foram realizados apenas com funcionários da Unipar.


## Gráfico 01


In [7]:
servico_frequencia = colunas_titular['Descricao Servico Sinistro'].value_counts()

top_10_servicos = servico_frequencia.head(10)

2. A primeira variável conta o número de ocorrências de cada valor na coluna Descricao Servico Sinistro
   
   <br>

    | Descrição do Serviço Ocorrências | Frequência |
    |-----------------------------------------------------------------|-------------|
    | CONSULTA CONSULTORIO (HORARIO NORMAL OU PREESTAB)               | 4545        |
    | SESSAO DE PSICOTERAPIA INDIVIDUAL POR PSICOLOGO                  | 958         |
    | HEMOG C/ CONT PLAQ OU FCS (ERITROGR, LEUCOG, PLAQ)               | 908         |
    | GLICOSE - PESQUISA E/OU DOSAGEM                                  | 797         |
    | CREATININA - PESQUISA E/OU DOSAGEM                               | 793         |
    | ...                                                             | ...         |
    | PROCEDIMENTO PADRONIZADO ANGIO-TC                                | 1           |
    | URETEROTOMIA INTERNA URETEROSCOPICA FLEXIVEL UNILATERAL          | 1           |
    | CORPO ESTRANHO - EXTRACAO ENDOSCOPICA                            | 1           |
    | PP HERPES SIMPLES - IGG - PESQUISA E/OU DOSAGEM                  | 1           |
    | AVAL. CLIN E ELETR. PCT PORT. MARCA-PAS/SINCR/DESF               | 1           |

<br>

3. Em seguida são selecionados os 10 serviços mais frequentes na coluna 'Descricao Servico Sinistro' e armazena na variável 'top_10_servicos'


In [ ]:
# Plotar o gráfico de barras
top_10_servicos.plot(kind='bar', color='skyblue')

# Adicionar título e rótulos aos eixos
plt.title('Top 10 Serviços Mais Utilizados')
plt.xlabel('Código do Serviço Sinistro')
plt.ylabel('Frequência de Uso')

# Exibir o gráfico
plt.show()

2. Criação de um gráfico para ilustrar a análise realizada com os dados. Um gráfico de barras será utilizado para visualizar os top 10 serviços mais utilizados pelos funcionários da Unipar.


## Gráfico 02


In [9]:
idade_frequencia = colunas_titular['Faixa-Etária Nova Sinistro'].value_counts()

1. Variável que conta o número de ocorrências de cada valor na coluna Faixa-Etária Nova Sinistro.

    <br>

    | Faixa-Etária Nova Sinistro | Frequência |
    |-------------------|---------------|
    | 59 anos ou mais   | 9742          |
    | 34 a 38 anos      | 6480          |
    | 39 a 43 anos      | 5293          |
    | 44 a 48 anos      | 4371          |
    | 49 a 53 anos      | 3751          |
    | 54 a 58 anos      | 3558          |
    | 29 a 33 anos      | 3185          |
    | 24 a 28 anos      | 2067          |
    | 19 a 23 anos      | 1062          |
    | 0 a 18 anos       | 14            |

In [ ]:
# Plotar o gráfico de barras
idade_frequencia.plot(kind='bar', color='skyblue')

# Adicionar título e rótulos aos eixos
plt.title('Faixa-Etária Colaboradores')
plt.xlabel('Faixas de idade')

# Exibir o gráfico
plt.show()

2. Criação de um gráfico para ilustrar a análise realizada com os dados. Um gráfico de barras será utilizado para visualizar a distribuição do número de sinistros por faixa etária.


## Gráfico 03


In [ ]:
plano_frequencia = colunas_titular['Descricao Plano Sinistro'].value_counts()
print(plano_frequencia)

1. Variável que conta o número de ocorrências de cada valor na coluna Descricao Plano Sinistro.

    <br>

    | Descrição do Plano Ocorrências | Frequência |
    |--------------------|-------------|
    | TQN2               | 17697       |
    | NP2X               | 2776        |
    | NP6X               | 1515        |
    | TNQ2               | 1223        |
    | TN1E               | 1117        |
    | TP8X               | 619         |
    | TNE1               | 437         |

In [ ]:
# Plotar o gráfico de barras
plano_frequencia.plot(kind='bar', color='skyblue')

# Adicionar título e rótulos aos eixos
plt.title('Planos Colaboradores')
plt.xlabel('Planos')

# Exibir o gráfico
plt.show()

2. Criação de um gráfico para ilustrar a análise realizada com os dados. Um gráfico de barras será utilizado para visualizar quais planos de saúde são mais utilizados.


# Pré-Processamento

Primeiro, vamos renomear todas as colunas para seguir as boas práticas

In [13]:
df = df.rename(columns={'Apolice Sinistro': 'apolice_sinistro','Codigo Empresa Sinistro': 'codigo_empresa_sinistro','Nome Empresa Sinistro': 'nome_empresa_sinistro','SEGURADO': 'segurado','Codigo Especialidade Sinistro': 'codigo_especialidade_sinistro','Elegibilidade Sinistro': 'elegibilidade_sinitro','Sexo Sinistro': 'sexo_sinistro','Faixa-Etária Nova Sinistro': 'faixa_etaria_sinistro','Descricao Plano Sinistro': 'descricao_plano_sinistro','Codigo Servico Sinistro': 'codigo_servico_sinistro','Descricao Servico Sinistro': 'descricao_servico_sinistro','Tipo Utilização Sinistro': 'tipo_utilizacao_sinistro','Dt Data Sinistro': 'data_sinistro','Codigo Prestador': 'codigo_prestador','Nome Prestador Sinistro': 'nome_prestador_sinistro','Valor Pago Sinistro': 'valor_pago_sinistro','Valor Usuario Sinistro': 'valor_usuario_sinistro','Quantidade': 'quantidade','No Ano Mes': 'ano_mes','Codigo Grupo Empresa': 'codigo_grupo_empresa','Nome Grupo Empresa': 'nome_grupo_empresa',})

Agora, precisamos filtrar nossa base de dados para conter apenas os Titulares

In [ ]:
df = df[df['elegibilidade_sinitro'] == 'TITULAR']
df['elegibilidade_sinitro'].value_counts()

In [15]:
pd.set_option('display.max_columns', None) # Configuracao para mostrar todas as colunas

## Tratamento de valores duplicados

Fazer a limpeza de valores duplicados é importantes pois valores duplicados podem prejudicar no aprendizado do nosso algoritmo. Para realizar a limpeza, vamos primeiro checar se existem valores duplicados e caso existirem, vamos excluir todos.

In [ ]:
duplicados = df[df.duplicated()]
print(len(duplicados))

Como podemos ver, existem 130 valores duplicados, o que vamos fazer é excluir todos para que assim eles não atrapalhem nosso algoritmo.

In [17]:
df = df.drop_duplicates()

## Tratamento de missing values

O comando df.isnull().sum() encontra todos os missing values. E como podemos ver, não existe nenhum valor null na nossa base de dados.

In [ ]:
df.isnull().sum()

Também ao olhar a coluna 'nome_prestador_sinistro', vemos que existem 3792 casos em que o valor é igual = "NAO INFORMADO". Como não podemos descobrir qual é o nome do prestador, vamos deixar do jeito que está.

In [ ]:
df['nome_prestador_sinistro'].value_counts()

Já esse comando é capaz de encontrar todos os valores = 0. Como podemos ver, Existem alguns valores importantes iguais a 0 e que não poderiam estar assim.

In [ ]:
zero_count = (df == 0).sum()
print(zero_count)

Para resolver os valores 0 na quantidade, vamos substituir eles pela moda da coluna, que é 1.

In [21]:
df['quantidade'] = df['quantidade'].replace(0, 1)

Fazendo uma outra análise, podemos ver que UNIPAR INDUPA DO BRASIL S.A e UNIPAR INDUPA DO BRASIL S.A. são a mesma empresa mas elas se diferem pelo ponto. Provavelmente um erro de digitação, esse é um erro que precisa ser tratado!

In [ ]:
df['nome_empresa_sinistro'].value_counts()

In [23]:
df['nome_empresa_sinistro'] = df['nome_empresa_sinistro'].replace('UNIPAR INDUPA DO BRASIL S.A.', 'UNIPAR INDUPA DO BRASIL S.A')

In [ ]:
df['nome_empresa_sinistro'].value_counts()

## Identificação de outliers e correção

As três colunas com possíveis outliers, são "valor_pago_sinistro", "valor_usuario_sinistro" e "quantidade". Isso porque todas as outras colunas numéricas são de identificação.

Como podemos ver, a coluna "valor_usuario_sinistro" não possui outliers, já que contém apenas o valor 0.

In [ ]:
df['valor_usuario_sinistro'].value_counts()

In [ ]:
sns.boxplot(data=df, y='valor_pago_sinistro')
plt.show()

In [ ]:
sns.boxplot(data=df, y='quantidade')
plt.show()

Como podemos ver, as duas colunas acima, possuem incontáveis outliers. Agora vamos tratá-los.

Basta observar os gráficos acima para entender que apesar de existirem milhares de outliers, muitos deles acontecem na mesma faixa e outros são casos raros, mesmo assim são números muito grandes. Por isso, vamos remover os valores em 'valor_pago_sinistro' que são maiores que 10.000 e os valores maiores que 10 em 'quantidade'.

Os valores maiores que 10.000 em 'valor_pago_sinistro' somam apenas 275 ocorrências. São poucas ocorrências e por isso vamor excluir elas, caso contrário, vão atrapalhar nosso algoritmo pois são outliers muito significativos.

Já os valores maiores que 10 em 'quantidade', somam apenas 9 ocorrências, pelos mesmos motivos, vamos excluir cada uma.

Para tratar desse problema, vamos criar uma cópia do nosso banco de dados e excluir esses valores do nosso novo banco de dados.

In [28]:
df_no_outliers = df.copy()

In [29]:
df_no_outliers = df_no_outliers[df_no_outliers['valor_pago_sinistro'] < 10000]
df_no_outliers = df_no_outliers[df_no_outliers['quantidade'] < 10]

In [ ]:
sns.boxplot(data=df_no_outliers, y='valor_pago_sinistro')
plt.show()

In [ ]:
sns.boxplot(data=df_no_outliers, y='quantidade')
plt.show()

Como podemos observar, ainda existem outliers, mas esses são valores que aconteceram muito e por isso possuem a tendência de acontecer novamente, logo, não vale a pena excluir esses valores da nossa base de dados.

Também é importante excluir as colunas que possuem apenas um único valor em todas as linhas porque assim vamos garantir vários benefícios, por exemplo:

1. Redução de Redundância
2. Simplificação do Modelo
3. Melhoria do Desempenho do Modelo
4. Redução de Overfitting
5. Economia de Espaço e Tempo

In [ ]:
columns_with_unique_values = [col for col in df.columns if df[col].nunique() == 1]
print(columns_with_unique_values)

In [33]:
df_cleaned = df_no_outliers.drop(columns=columns_with_unique_values)

## Codificação correta das variáveis de data

Uma coluna que será importante para nosso e que precisa ser tratada, é a coluna 'data_sinistro'. Como ela é uma coluna que contém datas, primeiro precisamos transformar os valores em *timestamp*, que é uma representação numérica que indica um ponto específico no tempo.

Para isso, vamos usar o comando abaixo, ele transformará nossa coluna para o padrão que desejamos.

In [34]:
df_cleaned['data_sinistro'] = pd.to_datetime(df['data_sinistro'], format='%d/%m/%Y')

Como nossos componentes temporais são relevantes para a análise, precisamos extrair o ano, mês e dia da nossa coluna para facilitar a identificação de sazionalidades e tendências anuais.

Após extrair os componentes temporais, vamos excluir a coluna 'data_sinistro' pois ela já não é mais necessária.

In [35]:
df_cleaned['dia_data_sinistro'] = df_cleaned['data_sinistro'].dt.day
df_cleaned['mes_data_sinistro'] = df_cleaned['data_sinistro'].dt.month
df_cleaned['ano_data_sinistro'] = df_cleaned['data_sinistro'].dt.year
df_cleaned = df_cleaned.drop(columns=['data_sinistro'])

Agora, vamos realizar a exclusão da coluna 'ano_mes' da nossa base de dados, já que não vamos utilizar ela para o treinamento do nosso modelo. Isso porque vemos como mais importante identificar sazonalidades no momento em que o segurado utilizou o plano de saúde, isso é mostrado nas colunas 'dia_data_sinistro', 'mes_data_sinistro' e 'ano_data_sinistro'. Já a coluna 'ano_mes', mostra apenas quando é lançada a fatura do sinistro para a empresa. Por essas razões, vamos excluir essa coluna. 

Duas colunas que também não serão utilizadas são: 'codigo_prestador' e 'nome_prestador_sinistro'. Essas colunas apenas identificam o prestador do sinistro e, portanto, não são relevantes para nossa análise, que visa encontrar insights relacionados aos sinistros em si.

In [36]:
df_cleaned = df_cleaned.drop(columns=['ano_mes', 'codigo_prestador', 'nome_prestador_sinistro'])

## Codificação correta das variáveis categóricas e normalização das variáveis numéricas

Para codificar corretamente as variáveis categóricas, primeiro, vamos colocar em uma array todas as colunas com variáveis categóricas.

In [ ]:
object_columns = [col for col in df_cleaned.columns if df_cleaned[col].dtype == 'object']
print(object_columns)

In [ ]:
for col in object_columns:
    print(df_cleaned[col].nunique())

Como podemos ver acima, 'nome_empresa_sinistro' e 'tipo_utilizacao_sinistro' possuem apenas 2 valores únicos. Como queremos que o algoritmo diferencie com clareza as empresas e tipos de utilização do sinistro, vamos utilizar o oneHotEncoder para codificar corretamente essas duas variáveis.

In [39]:
hot_encoder = OneHotEncoder(sparse_output=False)
label_encoder = LabelEncoder()
scaler = StandardScaler()

In [40]:
nome_empresa_sinistro_encoded_array = hot_encoder.fit_transform(df_cleaned[['nome_empresa_sinistro']])
nome_empresa_sinistro_encoded_dataframe = pd.DataFrame(nome_empresa_sinistro_encoded_array, columns=hot_encoder.get_feature_names_out(['nome_empresa_sinistro']))

In [41]:
tipo_utilizacao_sinistro_encoded_array = hot_encoder.fit_transform(df_cleaned[['tipo_utilizacao_sinistro']])
tipo_utilizacao_sinistro_encoded_dataframe = pd.DataFrame(tipo_utilizacao_sinistro_encoded_array, columns=hot_encoder.get_feature_names_out(['tipo_utilizacao_sinistro']))

In [42]:
df_cleaned.reset_index(drop=True, inplace=True)
nome_empresa_sinistro_encoded_dataframe.reset_index(drop=True, inplace=True)
tipo_utilizacao_sinistro_encoded_dataframe.reset_index(drop=True, inplace=True)

df_encoded = pd.concat([df_cleaned, nome_empresa_sinistro_encoded_dataframe, tipo_utilizacao_sinistro_encoded_dataframe], axis=1)
df_encoded = df_encoded.drop(columns=['nome_empresa_sinistro', 'tipo_utilizacao_sinistro'])
df_encoded = df_encoded.rename(columns={'nome_empresa_sinistro_UNIPAR CARBOCLORO S.A': 'nome_empresa_sinistro_unipar_carbocloro_s.a', 'nome_empresa_sinistro_UNIPAR INDUPA DO BRASIL S.A': 'nome_empresa_sinistro_unipar_indupa_do_brasil_s.a', 'tipo_utilizacao_sinistro_REDE': 'tipo_utilizacao_sinistro_rede', 'tipo_utilizacao_sinistro_REEMBOLSO': 'tipo_utilizacao_sinistro_reembolso',})

Agora, para terminar a codificação das variáveis categóricas, vamos utilizar o labelEncoder() que converte variáveis categóricas em variáveis numéricas. Pra isso, vamos criar uma array com as últimas colunas com variáveis categóricas e transformar elas com o labelEncoder.

In [ ]:
last_object_columns = [col for col in df_encoded.columns if df_encoded[col].dtype == 'object']
print(last_object_columns)

In [44]:
for col in last_object_columns:
    df_encoded[col] = label_encoder.fit_transform(df_encoded[col])

Agora para fazer a normalização das variáveis numéricas, as únicas colunas disponíveis para fazer isso seriam as colunas 'valor_pago_sinistro', 'quantidade', 'dia_data_sinistro', 'mes_data_sinistro', 'ano_data_sinistro' e 'codigo_servico_sinistro'.

Não vamos realizar a normalização das outras colunas numéricas porque já utilizamos o LabelEncoder ou OneHotEncoder.

É importante normalizar as variáveis numéricas já que muitos algoritmos são sensíveis à escala dos dados.

Para fazer a normalização das variáveis numéricas, vamos usar o StandardScaler, já que esse método é menos sensível à outliers.

In [45]:
valor_pago_sinistro_esc = scaler.fit_transform(df_cleaned[['valor_pago_sinistro']])
quantidade_esc = scaler.fit_transform(df_cleaned[['quantidade']])
dia_data_sinistro_esc = scaler.fit_transform(df_cleaned[['dia_data_sinistro']])
mes_data_sinistro_esc = scaler.fit_transform(df_cleaned[['mes_data_sinistro']])
ano_data_sinistro_esc = scaler.fit_transform(df_cleaned[['ano_data_sinistro']])
codigo_servico_sinistro_esc = scaler.fit_transform(df_cleaned[['codigo_servico_sinistro']])

valor_pago_sinistro_esc_dataframe = pd.DataFrame(valor_pago_sinistro_esc, columns=['valor_pago_sinistro_esc'])
quantidade_esc_dataframe = pd.DataFrame(quantidade_esc, columns=['quantidade_esc'])
dia_data_sinistro_esc_dataframe = pd.DataFrame(dia_data_sinistro_esc, columns=['dia_data_sinistro_esc'])
mes_data_sinistro_esc_dataframe = pd.DataFrame(mes_data_sinistro_esc, columns=['mes_data_sinistro_esc'])
ano_data_sinistro_esc_dataframe = pd.DataFrame(ano_data_sinistro_esc, columns=['ano_data_sinistro_esc'])
codigo_servico_sinistro_esc_dataframe = pd.DataFrame(codigo_servico_sinistro_esc, columns=['codigo_servico_sinistro_esc'])


In [46]:
df_encoded_esc = pd.concat([df_encoded, valor_pago_sinistro_esc_dataframe, quantidade_esc_dataframe, dia_data_sinistro_esc_dataframe, mes_data_sinistro_esc_dataframe, ano_data_sinistro_esc_dataframe, codigo_servico_sinistro_esc_dataframe], axis=1)
df_encoded_esc = df_encoded_esc.drop(columns=['valor_pago_sinistro', 'quantidade', 'dia_data_sinistro', 'mes_data_sinistro', 'ano_data_sinistro', 'codigo_servico_sinistro'])

In [ ]:
df_encoded.head(3)

In [ ]:
df_encoded_esc.head(3)

## Hipóteses

#### Hipótese 1: A faixa etária dos colaboradores pode ter uma correlação positiva com a sinistralidade do plano de saúde

&nbsp;&nbsp;&nbsp;&nbsp; É notável que, na maioria dos casos, pessoas em faixas etárias mais avançadas tendem a ter maiores necessidades médicas, resultando em um uso mais frequente do plano de saúde devido a consultas e visitas hospitalares. Caso os dados realmente demonstrem que colaboradores com idade mais avançada estão utilizando mais o plano de saúde, seria interessante considerar a implementação de um plano especializado, focado na prevenção e no tratamento de doenças crônicas

In [ ]:
# Agrupar por Faixa Etária e calcular o valor médio dos sinistros
media_sinistro_faixa_etaria = df.groupby('faixa_etaria_sinistro')['valor_pago_sinistro'].sum().reset_index()

soma_do_sinistro = media_sinistro_faixa_etaria.head(10)

media_sinistro_faixa_etaria

In [ ]:
# Plotar o gráfico de barras
soma_do_sinistro.plot(kind='bar', color='skyblue')

# Adicionar título e rótulos aos eixos
plt.title('Faixa Etária Comparada com Valor Pago de Sinistro')
plt.xlabel('Faixa Etária Sinistro')
plt.ylabel('Valor Pago Sinistro')

# Exibir o gráfico
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp; Diante dessa análise dos dados, concluímos que a faixa etária dos colaboradores, em conjunto com o valor pago pelos sinistros em cada tipo de consulta, indica que esses sinistros estão sendo utilizados predominantemente por colaboradores de faixa etária mais elevada dentro da empresa. Com isso, a UNIPAR pode considerar a criação de planos de apoio mais específicos para esses colaboradores.

#### Hipótese 2: O exame de Psicoterapia Individual por Psicólogo apresenta uma alta taxa de utilização por parte dos colaboradores.

&nbsp;&nbsp;&nbsp;&nbsp;Nos últimos anos, as empresas têm se preocupado cada vez mais com a saúde mental de seus funcionários, conscientizando-os sobre a importância desse aspecto, o que pode ter um impacto significativo no ambiente corporativo. Dessa forma, é esperado que a taxa de utilização de consultas psicológicas seja alta entre os colaboradores. Essa alta demanda pode indicar que as necessidades de saúde mental dos funcionários estão sendo reconhecidas e devidamente atendidas.

In [51]:
servico_frequencia = colunas_titular['Descricao Servico Sinistro'].value_counts()

top_10_servicos= servico_frequencia.head(10)

In [ ]:
# Plotar o gráfico de barras
top_10_servicos.plot(kind='bar', color='skyblue')

# Adicionar título e rótulos aos eixos
plt.title('Top 10 Serviços Mais Utilizados')
plt.xlabel('Descricao Serviço Sinistro')
plt.ylabel('Frequência de Uso')

# Exibir o gráfico
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp;Diante esse gráfico podemos ver que o segundo tipo de serviço de sinistro mais utilizado dentro da empresa UNIPAR é a sessão de psicoterapia, e com isso comprovamos nossa hipótese

#### Hipótese 3: O plano de sinistro mais utilizado pelos colaboradores foi o TQN2.

&nbsp;&nbsp;&nbsp;&nbsp;O plano TQN2 pode oferecer uma cobertura abrangente ou benefícios que são mais atrativos ou relevantes para os colaboradores em comparação com outros planos disponíveis pela empresa. Isso poderia incluir uma melhor cobertura para consultas, exames, terapias, ou uma rede credenciada mais extensa.

In [53]:
sinistro_frequencia = colunas_titular['Descricao Plano Sinistro'].value_counts()

top_10_descricoes= sinistro_frequencia.head(10)

In [ ]:
# Plotar o gráfico de barras
top_10_descricoes.plot(kind='bar', color='skyblue')

# Adicionar título e rótulos aos eixos
plt.title('Top 10 Planos de Sinistro Mais Utilizados')
plt.xlabel('Descricao Plano Sinistro')
plt.ylabel('Frequência de Uso')

# Exibir o gráfico
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp;Com esse gráfico, podemos analisar que é inevitável a diferença de quantidade de uso do plano TQN2, com isso podemos ainda mais aumentar a eficiência dessa nossa hipótese criada
 

### Hipótese 4: A faixa etária dos colaboradores que mais acionam o plano de Sinistro TQN2 é 34 a 38 anos

&nbsp;&nbsp;&nbsp;&nbsp;A faixa etária dos colaboradores que mais acionam o plano de Sinistro TQN2 é de 34 a 38 anos. Essa hipótese baseia-se na ideia de que, nessa fase da vida, os colaboradores podem estar enfrentando um aumento nas necessidades de saúde. A confirmação ou refutação dessa hipótese será explorada na análise do gráfico a seguir.

In [ ]:
dfTqn2 = df[df["descricao_plano_sinistro"] == "TQN2"]
tqn2PorFaixaEtariaSexo = dfTqn2.groupby(['faixa_etaria_sinistro', 'sexo_sinistro']).size().unstack(fill_value=0)

faixasEtarias = tqn2PorFaixaEtariaSexo.index
sexCounts = {
    'Masculino': tqn2PorFaixaEtariaSexo['M'].values,
    'Feminino': tqn2PorFaixaEtariaSexo['F'].values,
}
width = 0.6 

fig, ax = plt.subplots(figsize=(10, 7))
bottom = np.zeros(len(faixasEtarias))

for sexo, counts in sexCounts.items():
    p = ax.bar(faixasEtarias, counts, width, label=sexo, bottom=bottom)
    bottom += counts

    ax.bar_label(p, label_type='center')

ax.set_title('Utilização do Plano TQN2 por Faixa Etária e Sexo')
ax.set_xlabel('Faixa Etária')
ax.set_ylabel('Quantidade de Sinistros')

ax.legend(title='Sexo')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp;Com a análise do gráfico acima, podemos observar que dentro do plano mais utilizado os colaboradores da faixa etária de 34 a 38 anos acionam mais sinistros deste plano.

#### Hipótese 5: Existe uma tendência maior dos colaboradores da faixa etária de 59 anos ou mais acionarem mais o sinistro de Consulta no Consultório

&nbsp;&nbsp;&nbsp;&nbsp;Existe uma tendência maior dos colaboradores na faixa etária de 59 anos ou mais de acionar o sinistro de Consulta no Consultório com maior frequência. Essa hipotese baseia-se na ideia de que, com o avanço da idade, as necessidades de cuidados médicos regulares e acompanhamento contínuo aumentam, fazendo com que essa faixa etária busque mais frequentemente consultas médicas.

In [ ]:
df_faixa59ouMais = df[df["faixa_etaria_sinistro"] == "59 anos ou mais"]
frequenciaSinistroFaixa59ouMais = df_faixa59ouMais['descricao_servico_sinistro'].value_counts()
sinistrosFaixa59ouMais = frequenciaSinistroFaixa59ouMais.head(5)

#Plotar o gráfico
sinistrosFaixa59ouMais.plot(kind='pie', autopct='%1.1f%%', colors=['skyblue', 'lightgreen', 'lightcoral', 'orange', 'lightpink'])
plt.title('5 Serviços mais utilizados pelos colaboradores da faixa etária 59 anos ou mais')

plt.ylabel('')
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp;Após realizarmos a nossa analise gráfica podemos observar que dentre os serviços de sinistro mais utilizado para esta faixa etária é a CONSULTA CONSULTORIO (HORARIO NORMAL OU PRESTAB)

#### Hipótese 6: Existe uma tendência maior dos colaboradores da faixa etária de 59 anos ou mais acionarem mais o sinistro de Consulta no Consultório durante o mês de junho

&nbsp;&nbsp;&nbsp;&nbsp;Há uma tendência maior dos colaboradores da faixa etária de 59 anos ou mais de acionarem o sinistro de Consulta no Consultório especificamente durante o mês de junho. Essa hipótese está relacionada a fatores sazonais, como mudanças climáticas durante este mês.

In [ ]:
df_faixa59ouMais = df[df["faixa_etaria_sinistro"] == "59 anos ou mais"]
df_faixa59ouMais = df_faixa59ouMais[df_faixa59ouMais["descricao_servico_sinistro"] == "CONSULTA CONSULTORIO (HORARIO NORMAL OU PREESTAB)"]

df_faixa59ouMais['data_sinistro'] = pd.to_datetime(df_faixa59ouMais['data_sinistro'], format='%d/%m/%Y')
df_2023 = df_faixa59ouMais[df_faixa59ouMais['data_sinistro'].dt.year == 2023]
sinistros_por_mes_2023 = df_2023['data_sinistro'].dt.month.value_counts().sort_index()

sinistros_por_mes_2023_df = sinistros_por_mes_2023.reset_index()
sinistros_por_mes_2023_df.columns = ['Mês', 'Quantidade de Sinistros']

# Plotar o gráfico
plt.figure(figsize=(10, 6))
plt.plot(sinistros_por_mes_2023_df['Mês'], sinistros_por_mes_2023_df['Quantidade de Sinistros'], marker='o', linestyle='-', color='blue')

plt.title('Tendência de Sinistros por Mês em 2023')
plt.xlabel('Mês')
plt.ylabel('Quantidade de Sinistros')
plt.xticks(ticks=range(1, 13), labels=['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])

plt.grid(True)
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp;Realizamos uma análise da quantidade de acionamentos do serviço de sinistro Consulta no Consultório durante todos os meses de 2023, muito importante enfatizarmos que está análise foi realizada de acordo com as datas dos dados disponibilizados, portanto não é possível realizarmos uma anáise mais específica da sazonalidade ao passar dos anos. 

# Caso Não-Supervisionado

## Modelagem, features e explicação

### Modelagem do problema e clareza na explicação

O objetivo principal do nosso modelo é analisar e agrupar os dados fornecidos pela Unipar, utilizando técnicas de machine learning, especificamente clustering, para extrair insights importantes sobre os padrões de utilização do plano de saúde pelos colaboradores. Estamos utilizando aprendizagem não supervisionada, mais precisamente clustering, pelas seguintes razões:

1) Não temos categorias predefinidas de problemas de saúde ou padrões de utilização, tornando a aprendizagem não supervisionada a escolha mais apropriada.
2) Queremos descobrir padrões naturais nos dados de saúde sem impor categorias predefinidas, permitindo uma visão mais abrangente e potencialmente reveladora.
3) O clustering nos permite agrupar colaboradores com perfis de saúde e utilização do plano similares, revelando estruturas intrínsecas que podem não ser imediatamente aparentes nos relatórios convencionais.

O algoritmo de clustering que estamos empregando, vai nos ajudar a segmentar os dados em grupos distintos baseados em características similares de utilização do plano de saúde e perfis de saúde dos colaboradores.

Esperamos que este modelo resolva o problema em questão fornecendo um agrupamento capaz de revelar insights valiosos sobre os padrões e tendências nos dados de saúde. Com esses insights, a Unipar será capaz de:

1) Identificar segmentos distintos de colaboradores com necessidades de saúde similares.
2) Entender as características comuns dentro de cada grupo de saúde.
3) Desenvolver programas de saúde personalizados para cada segmento identificado.
4) Otimizar a alocação de recursos e a eficácia dos programas de saúde com base nas particularidades de cada grupo.

Por exemplo, o modelo pode revelar grupos de colaboradores com padrões similares de utilização do plano de saúde ou com condições de saúde semelhantes, permitindo que a Unipar desenvolva programas de saúde mais direcionados e eficazes, como programas para doenças crônicas, tabagismo, alcoolismo ou programas para gestantes.

### Escolha das features e justificativas

As features que escolhemos incluir em nosso modelo de clustering são:

'codigo_empresa_sinistro', 'codigo_especialidade_sinistro', 'sexo_sinistro', 'faixa_etaria_sinistro', 'descricao_plano_sinistro', 'descricao_servico_sinistro', 'nome_empresa_sinistro_unipar_carbocloro_s.a', 'nome_empresa_sinistro_unipar_indupa_do_brasil_s.a', 'tipo_utilizacao_sinistro_rede', 'tipo_utilizacao_sinistro_reembolso', 'valor_pago_sinistro_esc', 'quantidade_esc', 'dia_data_sinistro_esc', 'mes_data_sinistro_esc', 'ano_data_sinistro_esc', 'codigo_servico_sinistro_esc'

Justificativa para a escolha destas features:

1. Relevância para o problema: 
   Decidimos manter praticamente todas as colunas disponibilizadas pela Unipar porque cada uma delas fornece informações valiosas sobre os padrões de utilização do plano de saúde e as características dos colaboradores. Isso é crucial para nosso objetivo de criar programas de saúde personalizados e eficazes.

   - 'codigo_empresa_sinistro', 'nome_empresa_sinistro_unipar_carbocloro_s.a', 'nome_empresa_sinistro_unipar_indupa_do_brasil_s.a': 
   
      Estas features nos permitem identificar padrões específicos de cada empresa dentro do grupo Unipar.
   
   - 'codigo_especialidade_sinistro', 'descricao_servico_sinistro', 'codigo_servico_sinistro_esc': 
   
      Estas nos ajudam a entender quais tipos de serviços médicos são mais utilizados.
   
   - 'sexo_sinistro', 'faixa_etaria_sinistro': 
   
      Fundamentais para identificar padrões demográficos de saúde.
   
   - 'descricao_plano_sinistro': 
   
      Nos permite analisar se há diferenças significativas na utilização entre diferentes planos.
   
   - 'tipo_utilizacao_sinistro_rede', 'tipo_utilizacao_sinistro_reembolso': 
   
      Ajudam a entender como os colaboradores estão utilizando o plano (dentro ou fora da rede).
   
   - 'valor_pago_sinistro_esc', 'quantidade_esc': 
   
      Importantes para identificar padrões de custo e frequência de utilização.
   
   - 'dia_data_sinistro_esc', 'mes_data_sinistro_esc', 'ano_data_sinistro_esc': 
   
      Permitem identificar padrões sazonais ou tendências ao longo do tempo.

2. Pré-processamento e transformação:
   Para otimizar o desempenho do nosso algoritmo de clustering, aplicamos o seguinte pré-processamento:
   - Escalonamento: Utilizamos o StandardScaler para escalonar toda nossa base de dados. Isso é crucial para garantir que todas as features contribuam igualmente para o clustering, independentemente de suas escalas originais.

3. Adequação para aprendizagem não supervisionada:
   Estas features são adequadas para um modelo de aprendizagem não supervisionada, especificamente clustering, porque:
   
   a) Diversidade de informações: As features selecionadas cobrem uma ampla gama de aspectos relacionados à utilização do plano de saúde, desde características demográficas até detalhes específicos dos serviços utilizados.
   
   b) Potencial para revelar padrões ocultos: Ao combinar informações sobre o tipo de serviço, custo, frequência e características do usuário, nosso modelo pode identificar grupos de colaboradores com necessidades de saúde similares que não seriam evidentes em uma análise tradicional.
   
   c) Ausência de rótulos predefinidos: Como não temos categorias predefinidas de "perfis de saúde", a abordagem não supervisionada nos permite descobrir naturalmente esses perfis a partir dos dados.
   
   d) Escalabilidade: Ao escalonar os dados, garantimos que o algoritmo de clustering possa operar eficientemente, considerando todas as features de maneira equitativa.

Ao utilizar estas features em nosso modelo de clustering, esperamos identificar grupos significativos de colaboradores com padrões similares de utilização do plano de saúde. Isso permitirá que a Unipar desenvolva programas de saúde mais direcionados e eficazes, potencialmente melhorando a saúde dos colaboradores e otimizando os custos relacionados ao plano de saúde.

## Modelo e discussão

### Apresentar o modelo candidato 

O modelo candidato que escolhemos para este projeto é o K-Means, um algoritmo de clustering amplamente utilizado em aprendizagem não supervisionada. O K-Means é particularmente adequado para nosso caso devido à sua eficácia em lidar com grandes conjuntos de dados e sua capacidade de identificar padrões em dados multidimensionais, como os que temos em nossa base de utilização do plano de saúde.

Aqui, cito uma explicação do funcionamento do K-Means:

- Inicialização: O algoritmo começa escolhendo aleatoriamente K pontos no espaço de dados como centróides iniciais, onde K é o número de clusters que desejamos formar.
- Atribuição: Cada ponto de dados é atribuído ao centróide mais próximo, formando K clusters iniciais.
- Atualização: O centróide de cada cluster é recalculado como a média de todos os pontos atribuídos a ele.
- Repetição: Os passos 2 e 3 são repetidos iterativamente até que os centróides se estabilizem ou um número máximo de iterações seja atingido.
- Convergência: O algoritmo converge quando as atribuições não mudam mais.

Por essas razões, escolhemos o K-Means como nosso primeiro modelo!

Agora, vamos começar o treinamento do nosso primeiro modelo com o K-Means.

Primeiro, vamos criar uma base de dados chamada 'df_encoded_all_esc' para o treinamento do nosso modelo. Também, vamos excluir a coluna 'segurado' da nossa base de dados porque essa coluna é apenas uma forma de identificar os titulares, logo, essa informação não é importante para o treinamento do nosso algoritmo.

In [58]:
df_encoded_all_esc = df_encoded_esc.drop(columns=['segurado'])

Como vamos trabalhar com o K-Means, a distância entre cada dado é extremamente importante para o agrupamento e formação dos grupos. Por isso, vamos utilizar o StandardScaler para escalonar os dados e consequentemente diminuir a distância entre cada ponto de dados.

In [59]:
df_encoded_all_esc = scaler.fit_transform(df_encoded)

### Definição do K e justificativa

Para aplicar o algoritmo K-Means, precisamos definir o valor de K, que representa o número de clusters em que queremos dividir nossos dados. O K é um hiperparâmetro crucial que determina quantos grupos distintos o algoritmo deve identificar nos dados de utilização do plano de saúde da Unipar.

A escolha do valor ideal de K é fundamental, pois influencia diretamente a qualidade e a interpretabilidade dos resultados do clustering. Um K muito baixo pode resultar em clusters muito amplos e genéricos, perdendo nuances importantes nos padrões de utilização do plano de saúde. Por outro lado, um K muito alto pode levar a clusters excessivamente específicos, dificultando a identificação de padrões significativos e aumentando o risco de overfitting.
Para definir com precisão o número de clusters, vamos utilizar o Elbow Method. Este método é uma técnica heurística comum para determinar o número ideal de clusters (K) em um conjunto de dados.

O Elbow Method funciona da seguinte maneira:

1. Executamos o algoritmo K-Means para diferentes valores de K (por exemplo, de 1 a 10).
2. Para cada K, calculamos a soma das distâncias quadradas dentro do cluster (WCSS - Within-Cluster Sum of Squares).
3. Plotamos um gráfico de WCSS versus K.
4. Procuramos o "cotovelo" no gráfico - o ponto onde a diminuição da WCSS começa a nivelar, formando uma curva semelhante a um cotovelo.
O valor de K neste ponto de "cotovelo" é considerado o número ideal de clusters, pois representa um equilíbrio entre a compactação do cluster e o número de clusters.

Este método nos ajuda a encontrar um ponto de equilíbrio onde adicionar mais clusters não melhora significativamente a qualidade do agrupamento, permitindo-nos escolher um número de clusters que seja tanto eficaz quanto computacionalmente eficiente para nossa análise dos dados de saúde da Unipar.

In [60]:
elbow_method_all_encoded = []
for i in range(1,11):
	kmeans = KMeans(n_clusters=i,init='k-means++',random_state=42,max_iter=300)
	kmeans.fit(df_encoded_all_esc)
	elbow_method_all_encoded.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(x=range(1,11),y=elbow_method_all_encoded,marker='o',color='red')
plt.title('Number of Clusters (K)')
plt.xlabel('Inertia')
plt.ylabel('Elbow Method for Optimal')

Observando o gráfico, é possível observar que o ponto onde a diminuição da WCSS começa a nivelar, formando uma curva semelhante a um cotovelo se encontra no valor 4. Com essa informação, é possível concluir que o número ideal de Clusters (K) é 4, logo, vamos trabalhar com esse valor.

Com o código abaixo, vamos treinar nosso primeiro modelo:

In [62]:
kmeans_esc_all = KMeans(n_clusters=4,init='k-means++',random_state=42,max_iter=300)
all_esc_kmeans = kmeans_esc_all.fit(df_encoded_all_esc)

In [63]:
labels = all_esc_kmeans.labels_

In [ ]:
pca_esc = PCA(n_components=2)
reduced_data = pca_esc.fit_transform(df_encoded_all_esc)

explained_variance_ratio = pca_esc.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

print(f"Variance explained by the first two principal components: {cumulative_variance_ratio[1]:.2%}")

Para visualizar com qualidade como o agrupamento foi feito, criamos um gráfico em 2D:

In [ ]:
plt.figure(figsize=(12, 9))

scatter = plt.scatter(reduced_data[:, 0], reduced_data[:, 1], 
                      c=labels, cmap='viridis')

plt.xlabel(f'First PC ({explained_variance_ratio[0]:.2%} variance)')
plt.ylabel(f'Second PC ({explained_variance_ratio[1]:.2%} variance)')
plt.title('K-means Clustering (4 clusters) - 2D PCA Visualization')

plt.colorbar(scatter)

legend1 = plt.legend(*scatter.legend_elements(),
                    loc="upper right", title="Clusters")
plt.gca().add_artist(legend1)

plt.show()

In [ ]:
pca_esc_3d = PCA(n_components=3)
reduced_data_3d = pca_esc_3d.fit_transform(df_encoded_all_esc)

explained_variance_ratio = pca_esc_3d.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

print(f"Variance explained by the first three principal components: {cumulative_variance_ratio[2]:.2%}")

Também criamos um gráfico em 3D:

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(reduced_data_3d[:, 0], reduced_data_3d[:, 1], reduced_data_3d[:, 2], 
                     c=labels, cmap='viridis')

ax.set_xlabel(f'First PC ({explained_variance_ratio[0]:.2%} variance)')
ax.set_ylabel(f'Second PC ({explained_variance_ratio[1]:.2%} variance)')
ax.set_zlabel(f'Third PC ({explained_variance_ratio[2]:.2%} variance)')
ax.set_title('K-means Clustering (4 clusters) - 3D PCA Visualization')

plt.colorbar(scatter)

plt.show()

Visualizando os dois gráficos, é possível observar que existe uma divisão extremamente harmoniosa em cada grupo. Esse é um bom sinal, pois mostra que o nosso algoritmo K-Means foi capaz de identificar cada grupo com facilidade. Agora, vamos criar uma coluna em nossa base de dados para identificar como cada segurado está dividido.

In [ ]:
df_k_means_esc_all = df_cleaned.copy()
df_k_means_esc_all['group'] = labels
df_k_means_esc_all.head(3)

Agora, criaremos quatro base de dados filtrando cada grupo, isso será importante porque agora nosso trabalho é entender como o K-Means realizou a divisão de cada grupo, assim, podemos extrair insights significativos de cada grupo. Logo abaixo, iremos realizar uma análise profunda em cada grupo e realizar a discussão sobre os resultados do modelo.

### Análise Exploratória dos Clusters do modelo

In [ ]:
df_k_means_group_0 = df_k_means_esc_all[df_k_means_esc_all['group'] == 0]
df_k_means_group_1 = df_k_means_esc_all[df_k_means_esc_all['group'] == 1]
df_k_means_group_2 = df_k_means_esc_all[df_k_means_esc_all['group'] == 2]
df_k_means_group_3 = df_k_means_esc_all[df_k_means_esc_all['group'] == 3]

# Contagem de elementos em cada cluster
print("Número de elementos em cada grupo:")
print(len(df_k_means_group_0), 
      len(df_k_means_group_1), 
      len(df_k_means_group_2), 
      len(df_k_means_group_3))

# Estatísticas descritivas dos 4 clusters
for group in range(4): 
    print(f"\nGroup {group} Statistics:")
    group_data = df_k_means_esc_all[df_k_means_esc_all['group'] == group]
    

    print(group_data.describe())
    
    # Frequências das variáveis categóricas
    for col in ['faixa_etaria_sinistro', 'descricao_plano_sinistro', 'tipo_utilizacao_sinistro']:
        print(f"\n{col} distribution in Group {group}:")
        print(group_data[col].value_counts())


Essa análise exploratória de todas as colunas para cada um dos clusters, construimos gráficos sobre o agrupamento dos clusters realizados pelo modelo K-means para obtermos uma melhor visualização dos agrupamentos.

In [ ]:
clusters_data = {
    'Cluster': ['Cluster 0', 'Cluster 1', 'Cluster 2', 'Cluster 3'],
    'Registros': [7038, 14738, 13461, 3671],
}

df_clusters = pd.DataFrame(clusters_data)

plt.figure(figsize=(10, 6))
plt.bar(df_clusters['Cluster'], df_clusters['Registros'], color='lightgreen')
plt.title('Quantidade de Registros por Cluster')
plt.xlabel('Clusters')
plt.ylabel('Quantidade de Registros')
plt.show()

Neste gráfico podemos visualizar a dispersão da quantidade de registros por cluster separado utilizando o modelo K-means.

In [ ]:
clusters = [0, 1, 2, 3]
sinistroCounts = {
    'REDE': [],
    'REEMBOLSO': []
}
for cluster in clusters:
    df_cluster = df_k_means_esc_all[df_k_means_esc_all['group'] == cluster]
    counts = df_cluster['tipo_utilizacao_sinistro'].value_counts()    
    sinistroCounts['REDE'].append(counts.get('REDE', 0))
    sinistroCounts['REEMBOLSO'].append(counts.get('REEMBOLSO', 0))

width = 0.4
x = np.arange(len(clusters))

fig, ax = plt.subplots(figsize=(10, 7))

colors = {
    'REDE': 'lightgreen',
    'REEMBOLSO': 'purple'
}

for tipo_sinistro, counts in sinistroCounts.items():
    p = ax.bar(x, counts, width, label=tipo_sinistro, color=colors[tipo_sinistro])

ax.set_title('Utilização do Sinistro por Cluster (REDE vs REEMBOLSO)')
ax.set_xlabel('Clusters')
ax.set_ylabel('Quantidade de Sinistros')
ax.set_xticks(x)
ax.set_xticklabels([f'Cluster {i}' for i in clusters])

ax.legend(title='Tipo de Sinistro')

plt.tight_layout()
plt.show()

Neste gráfico podemos observar a quantidade do tipo de utilização do sinistro por cluster, e um ponto muito interessante que podemos observar é que os três primeiros clusters só agruparam registros de utilização do tipo REDE, enquanto o quarto cluster separou somente registros do tipo REEMBOLSO.

In [ ]:
clusters = [0, 1, 2, 3]

valorPagoPorCluster = []

for cluster in clusters:
    df_cluster = df_k_means_esc_all[df_k_means_esc_all['group'] == cluster]
    valor_total = df_cluster['valor_pago_sinistro'].sum()
    valorPagoPorCluster.append(valor_total)

colors = ['yellow', 'skyblue', 'lightgreen', 'purple']

width = 0.6
x = np.arange(len(clusters))

fig, ax = plt.subplots(figsize=(10, 7))

p = ax.bar(x, np.array(valorPagoPorCluster)[clusters], width, color=[colors[i] for i in clusters])

ax.bar_label(p, fmt='%.2f', label_type='edge')

ax.set_title('Soma do Valor Pago por Cluster')
ax.set_xlabel('Clusters')
ax.set_ylabel('Valor Pago (R$)')
ax.set_xticks(x)
ax.set_xticklabels([f'Cluster {i}' for i in clusters])

plt.tight_layout()
plt.show()

Neste gráfico podemos observar a soma do valor pago para cada cluster, com isso podemos observar que o Cluster 3 possui majoritariamente valores mais custosos para a empresa, ao compararmos o valor com a quantidade de registros por cluster, desa forma podemos concluir que os serviços do tipo de utilização REEMBOLSO é mais custoso para a empresa.

Após construirmos estes gráficos sobre as informações gerais pdos clusters, construimos um gráfico para cada um dos clusters a fim de identificarmos visualmente a disperção da faixa etaria em cada um dos clusters.

In [ ]:
contagemFaixaEtaria = []

df_cluster = df_k_means_esc_all[df_k_means_esc_all['group'] == 0]

faixas_etarias_presentes = sorted(df_cluster['faixa_etaria_sinistro'].unique())

for faixa in faixas_etarias_presentes:
    contagem = df_cluster[df_cluster['faixa_etaria_sinistro'] == faixa].shape[0]
    contagemFaixaEtaria.append(contagem)

width = 0.6
x = np.arange(len(contagemFaixaEtaria))

fig, ax = plt.subplots(figsize=(5, 7))

p = ax.bar(x, np.array(contagemFaixaEtaria), width, color='yellow')

ax.bar_label(p, fmt='%d', label_type='edge')

ax.set_title('Contagem de Sinistros por Faixa Etária no Cluster 0')
ax.set_xlabel('Faixa Etária (Anos)')
ax.set_ylabel('Contagem de Sinistros')
ax.set_xticks(x)
ax.set_xticklabels([f'{faixa}' for faixa in faixas_etarias_presentes])

plt.tight_layout()
plt.show()


Podemos observar que o cluster 0 agrupou somente sinistros acionados por colaboradores mais velhos das faixas 54 a 58 anos e 59 ou mais.

In [ ]:
contagemFaixaEtaria = []

df_cluster = df_k_means_esc_all[df_k_means_esc_all['group'] == 1]

faixas_etarias_presentes = sorted(df_cluster['faixa_etaria_sinistro'].unique())

for faixa in faixas_etarias_presentes:
    contagem = df_cluster[df_cluster['faixa_etaria_sinistro'] == faixa].shape[0]
    contagemFaixaEtaria.append(contagem)

width = 0.6
x = np.arange(len(contagemFaixaEtaria))

fig, ax = plt.subplots(figsize=(12, 7))

p = ax.bar(x, np.array(contagemFaixaEtaria), width, color='skyblue')

ax.bar_label(p, fmt='%d', label_type='edge')

ax.set_title('Contagem de Sinistros por Faixa Etária no Cluster 1')
ax.set_xlabel('Faixa Etária (Anos)')
ax.set_ylabel('Contagem de Sinistros')
ax.set_xticks(x)
ax.set_xticklabels([f'{faixa}' for faixa in faixas_etarias_presentes])

plt.tight_layout()
plt.show()

Neste gráfico podemos observar que o Cluster 1 agrupou faixas etárias mais dispersas abrangendo todas as faixas etárias, porém em sua maoria adultos de 34 a 38 anos.

In [ ]:
contagemFaixaEtaria = []

df_cluster = df_k_means_esc_all[df_k_means_esc_all['group'] == 2]

faixas_etarias_presentes = sorted(df_cluster['faixa_etaria_sinistro'].unique())

for faixa in faixas_etarias_presentes:
    contagem = df_cluster[df_cluster['faixa_etaria_sinistro'] == faixa].shape[0]
    contagemFaixaEtaria.append(contagem)

width = 0.6
x = np.arange(len(contagemFaixaEtaria))

fig, ax = plt.subplots(figsize=(12, 7))

p = ax.bar(x, np.array(contagemFaixaEtaria), width, color='lightgreen')

ax.bar_label(p, fmt='%d', label_type='edge')

ax.set_title('Contagem de Sinistros por Faixa Etária no Cluster 2')
ax.set_xlabel('Faixa Etária (Anos)')
ax.set_ylabel('Contagem de Sinistros')
ax.set_xticks(x)
ax.set_xticklabels([f'{faixa}' for faixa in faixas_etarias_presentes])

plt.tight_layout()
plt.show()

Assim como o Cluster anterior o cluster 2 agrupou faixas etárias diversificadas, porém na maioria dos registros temos adultos de 34 a 38 anos

In [ ]:
contagemFaixaEtaria = []

df_cluster = df_k_means_esc_all[df_k_means_esc_all['group'] == 3]

faixas_etarias_presentes = sorted(df_cluster['faixa_etaria_sinistro'].unique())

for faixa in faixas_etarias_presentes:
    contagem = df_cluster[df_cluster['faixa_etaria_sinistro'] == faixa].shape[0]
    contagemFaixaEtaria.append(contagem)

width = 0.6
x = np.arange(len(contagemFaixaEtaria))

fig, ax = plt.subplots(figsize=(12, 7))

p = ax.bar(x, np.array(contagemFaixaEtaria), width, color='purple')

ax.bar_label(p, fmt='%d', label_type='edge')

ax.set_title('Contagem de Sinistros por Faixa Etária no Cluster 3')
ax.set_xlabel('Faixa Etária (Anos)')
ax.set_ylabel('Contagem de Sinistros')
ax.set_xticks(x)
ax.set_xticklabels([f'{faixa}' for faixa in faixas_etarias_presentes])

plt.tight_layout()
plt.show()

Assim como os clusters anteriores podemos observar neste gráfico que o Cluster 3 agrupou faixas etárias diversas, porém em sua maioria colaboradores de 34 a 38 anos.

### Conclusão do resultado da análise exploratória dos Clusters

Após realizarmos a análise exploratória e a construção dos gáficos anteriores dos Clusters conseguimos desenvolver algumas conclusões valiosas sobre a divisão dos 4 clusters.

Cluster 0: O Cluster 0 agrupa majoritariamente segurados de 59 anos ou mais com sinistros de menor valor, predominantemente na rede credenciada (REDE). Este grupo se beneficia de programas de saúde focados na prevenção e no controle de doenças crônicas, como acompanhamento médico regular e incentivo a atividades físicas, reduzindo o risco de complicações futuras e otimizando os custos dos planos de saúde.

Cluster 1: O Cluster 1 agrupa em maioria segurados de 34 a 38 anos, com sinistros de custo médio, também utilizando a REDE. Este grupo demanda programas de saúde ocupacional e de bem-estar, que incentivem a prevenção de doenças e o monitoramento da saúde mental e física, promovendo a produtividade e reduzindo os custos com sinistros.

Cluster 2: O Cluster 2, semelhante ao Cluster 1, apresenta sinistros de menor valor e maioria de segurados na faixa etária de 34 a 38 anos. Programas de prevenção e check-ups regulares seriam importates para manter os custos baixos e evitar o desenvolvimento de condições crônicas, além de promover hábitos saudáveis por meio de incentivos à prática de atividades físicas e uma boa nutrição.

Cluster 3: O Cluster 3 agrupa segurados que utilizam reembolso (REEMBOLSO), resultando em sinistros de alto valor. Para este grupo, seria importante implementar programas de gestão de custos, incentivando o uso de sinistros de REDE e educando os segurados sobre a importância de evitar procedimentos caros fora da rede.

# Novos Modelos de Clustering e Métricas de Avaliação

Nesta etapa do projeto, vamos expandir nossa análise de clustering testando três novos modelos além do K-means: DBSCAN, Gaussian Mixture Models (GMM) e HDBSCAN. Cada um desses modelos oferece uma abordagem única para a tarefa de clustering, permitindo-nos explorar diferentes perspectivas sobre nossos dados.

## Novos Modelos de Clustering

### 1. DBSCAN (Density-Based Spatial Clustering of Applications with Noise)
O DBSCAN é um algoritmo baseado em densidade que agrupa pontos que estão próximos uns dos outros no espaço de dados. Ele é particularmente eficaz na identificação de clusters de formato arbitrário e na detecção de outliers.

### 2. Gaussian Mixture Models (GMM)
GMM é um modelo probabilístico que assume que os dados são gerados a partir de uma mistura de um número finito de distribuições gaussianas com parâmetros desconhecidos. Este modelo é flexível e pode capturar clusters com formas elípticas.

### 3. HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise)
HDBSCAN é uma extensão do DBSCAN que cria uma hierarquia de clusters baseados em densidade. Ele oferece maior flexibilidade em comparação ao DBSCAN, sendo capaz de identificar clusters de densidades variadas.

## Importância de Testar Vários Modelos

Testar múltiplos modelos de clustering é vital por várias razões:

1. Diferentes conjuntos de dados têm estruturas diferentes, e nem todos os algoritmos são igualmente eficazes para todos os tipos de dados.
2. Cada modelo tem suas próprias suposições e vieses, e comparar vários modelos nos ajuda a entender melhor a estrutura subjacente dos dados.
3. A comparação de modelos pode revelar insights que não seriam aparentes ao usar apenas um único algoritmo.
4. Isso nos permite escolher o modelo mais apropriado para nosso conjunto de dados específico.

## Métricas de Avaliação

Para comparar o desempenho dos diferentes modelos, utilizaremos três métricas de avaliação:

### 1. Silhouette Score
- **Funcionamento**: Mede o quão similar um objeto é ao seu próprio cluster em comparação com outros clusters.
- **Interpretação**: Varia de -1 a 1. Valores próximos a 1 indicam que o objeto está bem correspondido ao seu próprio cluster e mal correspondido aos clusters vizinhos. Valores próximos a 0 indicam que o objeto está na fronteira entre dois clusters. Valores negativos sugerem que o objeto pode ter sido atribuído ao cluster errado.

### 2. Calinski-Harabasz Index
- **Funcionamento**: Também conhecido como Critério de Razão de Variância, este índice é definido como a razão entre a dispersão entre clusters e a dispersão dentro dos clusters.
- **Interpretação**: Quanto maior o valor do índice, melhor é a definição dos clusters. Um valor mais alto indica que os clusters são densos e bem separados.

### 3. Davies-Bouldin Index
- **Funcionamento**: Calcula a similaridade média entre cada cluster e seu cluster mais similar, onde a similaridade é a razão da distância dentro do cluster pela distância entre clusters.
- **Interpretação**: Valores mais baixos indicam melhor clustering. Um valor baixo indica que os clusters são compactos e bem separados uns dos outros.

Ao utilizar estas métricas em conjunto, poderemos obter uma visão abrangente da qualidade dos clusters produzidos por cada modelo, permitindo-nos fazer uma comparação objetiva e informada entre eles.

## Hyperparameter Tuning

### O que é Hyperparameter Tuning?
Hyperparameter Tuning é o processo de otimização dos parâmetros de um modelo de machine learning que não são aprendidos diretamente dos dados durante o treinamento. Esses parâmetros, chamados de hiperparâmetros, controlam o comportamento do algoritmo e podem ter um impacto significativo no desempenho do modelo.

### Importância do Hyperparameter Tuning
1. **Melhoria de Desempenho**: Ajustar os hiperparâmetros pode levar a melhorias significativas no desempenho do modelo.
2. **Adaptação ao Problema**: Permite adaptar o modelo às características específicas do conjunto de dados e do problema em questão.
3. **Evitar Overfitting/Underfitting**: Ajuda a encontrar o equilíbrio certo entre a complexidade do modelo e sua capacidade de generalização.
4. **Otimização de Recursos**: Pode ajudar a otimizar o uso de recursos computacionais, encontrando configurações eficientes.

### ParameterGrid para Hyperparameter Tuning

Para realizar o Hyperparameter Tuning, utilizaremos o ParameterGrid da biblioteca scikit-learn.

#### O que é ParameterGrid?
ParameterGrid é uma ferramenta que gera todas as combinações de parâmetros especificados em um dicionário. Ele é particularmente útil para a busca exaustiva de hiperparâmetros, também conhecida como Grid Search.

#### Como funciona o ParameterGrid:
1. **Definição**: Você define um dicionário onde as chaves são os nomes dos hiperparâmetros e os valores são listas ou arrays de valores possíveis para cada hiperparâmetro.

2. **Geração de Combinações**: O ParameterGrid gera todas as combinações possíveis dos valores especificados para cada hiperparâmetro.

3. **Iteração**: Você pode iterar sobre essas combinações, testando cada uma delas no seu modelo.

4. **Avaliação**: Para cada combinação, o modelo é treinado e avaliado, permitindo identificar a combinação que oferece o melhor desempenho.

Ao utilizar o ParameterGrid para Hyperparameter Tuning, seremos capazes de explorar sistematicamente diferentes configurações para cada um dos nossos modelos de clustering, garantindo que encontremos as melhores configurações possíveis para o nosso conjunto de dados específico.

Primeiro, antes de começar a criar novos modelos. Vamos criar uma função extremamente importante que receberá a base de dados e o agrupamento realizado pelo modelo nessa base de dados. A função `evaluate_clustering` retornará as três métricas que vamos trabalhar e que foram explicadas acima: `silhouette, calinski, davies`

In [77]:
def evaluate_clustering(X, labels):
    if len(np.unique(labels)) < 2:
        return -np.inf, -np.inf, np.inf
    
    silhouette = silhouette_score(X, labels)
    calinski = calinski_harabasz_score(X, labels)
    davies = davies_bouldin_score(X, labels)
    return silhouette, calinski, davies

Agora, começaremos a treinar novos modelos para no final, comparar as métricas entre geradas e identificar os melhores modelos para nosso projeto.

### DBSCAN 

Aqui criaremos uma função que receberá os a base de dados para o DBSCAN treinar, junto com seus hiperparâmetros. Essa função é importante porque assim poderemos treinar vários modelos, testar os hiperparâmetros com facilidade e decidir quais são os melhores para nosso caso.

In [78]:
def apply_dbscan(X, eps, min_samples):
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(X)
    return labels

Aqui criamos nosso a lógica para realizar o Hyperparameter Tuning e encontrar os melhores hiperparâmetros.

In [ ]:
param_grid_dbscan = {
    'eps': np.arange(0.1, 2.1, 0.1),
    'min_samples': range(2, 11)
}

best_score_dbscan = -np.inf
best_params_dbscan = None
results_dbscan = []

for params in ParameterGrid(param_grid_dbscan):
    labels = apply_dbscan(df_encoded_all_esc, params['eps'], params['min_samples'])
    silhouette, calinski, davies = evaluate_clustering(df_encoded_all_esc, labels)
    
    results_dbscan.append({
        'eps': params['eps'],
        'min_samples': params['min_samples'],
        'silhouette': silhouette,
        'calinski': calinski,
        'davies': davies
    })
    
    if silhouette > best_score_dbscan:
        best_score_dbscan = silhouette
        best_params_dbscan = params
    
    print(f"eps: {params['eps']}, min_samples: {params['min_samples']}, silhouette: {silhouette:.4f}")

print("\nBest parameters:")
print(best_params_dbscan)
print(f"Best silhouette score: {best_score_dbscan:.4f}")

Como o código acima demora uma eternidade para executar, aqui está o que estamos buscando:

**Resultados do modelo DBSCAN**

| Output    | Valor                |
|--------------|----------------------|
| eps          | 2.0                  |
| min_samples  | 7                    |
| silhouette   | 0.19251426832951582  |
| calinski     | 912.7302215995092    |
| davies       | 1.4955838623050175   |

Esta tabela apresenta os resultados obtidos com o modelo DBSCAN, incluindo os hiperparâmetros utilizados (eps e min_samples) e as métricas de avaliação (silhouette score, índice Calinski-Harabasz e índice Davies-Bouldin).

Agora, vamos criar um agrupamento utilzando o DBSCAN com os melhores hiperparâmetros encontrados. Depois vamos análisar em gráfico 2D e 3D como ficou o agrupamento e mais tarde vamos análisar as métricas para decidir o melhor entre todos nossos modelos.

In [ ]:
dbscan_with_best_labels = apply_dbscan(df_encoded_all_esc, min_samples=7, eps=2)

In [ ]:
# 2D
plt.figure(figsize=(10, 8))
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=dbscan_with_best_labels, cmap='viridis')
plt.title(f"DBSCAN Clustering 2D (eps={best_params_dbscan['eps']}, min_samples={best_params_dbscan['min_samples']})")
plt.colorbar(label='Cluster')
plt.show()

# 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(reduced_data_3d[:, 0], reduced_data_3d[:, 1], reduced_data_3d[:, 2], c=dbscan_with_best_labels, cmap='viridis')
ax.set_title(f"DBSCAN Clustering 3D (eps={best_params_dbscan['eps']}, min_samples={best_params_dbscan['min_samples']})")
plt.colorbar(scatter, label='Cluster')
plt.show()

Agora, criamos várias visualizações para analisar os resultados do algoritmo DBSCAN com diferentes combinações de parâmetros. O código gera quatro gráficos distintos, cada um oferecendo insights sobre como os parâmetros do DBSCAN afetam as métricas de avaliação do clustering.

Os primeiros três gráficos são plotados lado a lado e mostram como as três métricas de avaliação (Silhouette Score, Calinski-Harabasz Score e Davies-Bouldin Score) variam em função do parâmetro `eps` para diferentes valores de `min_samples`:

1. **Silhouette Score vs. Parameters**: 
   - Mostra como o Silhouette Score varia com `eps` para cada valor de `min_samples`.
   - Um Silhouette Score mais alto indica melhor definição dos clusters.

2. **Calinski-Harabasz Score vs. Parameters**: 
   - Ilustra a variação do índice Calinski-Harabasz em relação a `eps` para cada `min_samples`.
   - Valores mais altos do índice Calinski-Harabasz sugerem clusters mais bem definidos.

3. **Davies-Bouldin Score vs. Parameters**: 
   - Apresenta como o índice Davies-Bouldin muda com `eps` para diferentes `min_samples`.
   - Um índice Davies-Bouldin mais baixo indica uma melhor separação entre clusters.

Cada linha nesses gráficos representa um valor específico de `min_samples`, permitindo comparar facilmente como diferentes combinações de parâmetros afetam o desempenho do clustering.

4. O quarto gráfico é um gráfico de dispersão que oferece uma visão mais holística da relação entre os parâmetros do DBSCAN e o Silhouette Score:

- O eixo x representa o valor de `eps`.
- O eixo y representa o valor de `min_samples`.
- Cada ponto no gráfico é colorido de acordo com o Silhouette Score correspondente.
- Uma barra de cores ao lado do gráfico indica a escala do Silhouette Score.

Este gráfico permite visualizar rapidamente quais combinações de `eps` e `min_samples` resultam em melhores Silhouette Scores, indicados por cores mais quentes na escala de cores.

In [ ]:
results_df = pd.DataFrame(results_dbscan)

plt.figure(figsize=(15, 5))

# Silhouette Score
plt.subplot(131)
for min_samples in results_df['min_samples'].unique():
    data = results_df[results_df['min_samples'] == min_samples]
    plt.plot(data['eps'], data['silhouette'], label=f'min_samples={min_samples}', marker='o')
plt.xlabel('eps')
plt.ylabel('Silhouette Score')
plt.legend(title='min_samples', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('Silhouette Score vs. Parameters')

# Calinski-Harabasz Score
plt.subplot(132)
for min_samples in results_df['min_samples'].unique():
    data = results_df[results_df['min_samples'] == min_samples]
    plt.plot(data['eps'], data['calinski'], label=f'min_samples={min_samples}', marker='o')
plt.xlabel('eps')
plt.ylabel('Calinski-Harabasz Score')
plt.legend(title='min_samples', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('Calinski-Harabasz Score vs. Parameters')

# Davies-Bouldin Score
plt.subplot(133)
for min_samples in results_df['min_samples'].unique():
    data = results_df[results_df['min_samples'] == min_samples]
    plt.plot(data['eps'], data['davies'], label=f'min_samples={min_samples}', marker='o')
plt.xlabel('eps')
plt.ylabel('Davies-Bouldin Score')
plt.legend(title='min_samples', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('Davies-Bouldin Score vs. Parameters')

plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 8))
scatter = plt.scatter(results_df['eps'], results_df['min_samples'], 
                      c=results_df['silhouette'], cmap='viridis', 
                      s=50, alpha=0.6)
plt.colorbar(scatter, label='Silhouette Score')
plt.xlabel('eps')
plt.ylabel('min_samples')
plt.title('DBSCAN Parameters vs. Silhouette Score')
plt.tight_layout()
plt.show()

Depois de criar nossos novos modelos, vamos comparar todas as métricas! Agora, vamos repetir todo o processo que fizemos acima com nosso segundo modelo, o Gaussian Mixture Models

### Gaussian Mixture Models 

In [ ]:
def apply_gaussian(X, n_components, covariance_type):
    gmm = GaussianMixture(n_components=n_components, covariance_type=covariance_type, random_state=42)
    labels = gmm.fit_predict(X)
    return labels

In [ ]:
param_grid_gaussian = {
    'n_components': range(2, 11),
    'covariance_type': ['full', 'tied', 'diag', 'spherical']
}

best_score_gaussian = -np.inf
best_params_gaussian = None
results_gaussian = []

for params in ParameterGrid(param_grid_gaussian):
    labels = apply_gaussian(df_encoded_all_esc, params['n_components'], params['covariance_type'])
    silhouette, calinski, davies = evaluate_clustering(df_encoded_all_esc, labels)
    
    results_gaussian.append({
        'n_components': params['n_components'],
        'covariance_type': params['covariance_type'],
        'silhouette': silhouette,
        'calinski': calinski,
        'davies': davies
    })
    
    if silhouette > best_score_gaussian:
        best_score_gaussian = silhouette
        best_params_gaussian = params
    
    print(f"n_components: {params['n_components']}, covariance_type: {params['covariance_type']}, silhouette: {silhouette:.4f}")

print("\nBest parameters:")
print(best_params_gaussian)
print(f"Best silhouette score: {best_score_gaussian:.4f}")

Como o código acima demora uma eternidade para executar, aqui está o que estamos buscando:

**Resultados do modelo Gaussian Mixture**

| Output         | Valor                |
|----------------|----------------------|
| n_components   | 4                    |
| covariance_type| full                 |
| silhouette     | 0.2253971313441317   |
| calinski       | 8185.180301875231    |
| davies         | 1.7107242518807995   |

Esta tabela apresenta os resultados obtidos com o modelo Gaussian Mixture, incluindo os hiperparâmetros utilizados (n_components e covariance_type) e as métricas de avaliação (silhouette score, índice Calinski-Harabasz e índice Davies-Bouldin).

In [ ]:
gaussian_with_best_labels = apply_gaussian(df_encoded_all_esc, n_components=4, covariance_type='full')

In [ ]:
# 2D
plt.figure(figsize=(10, 8))
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=gaussian_with_best_labels, cmap='viridis')
plt.title(f"GMM Clustering 2D (n_components={best_params_gaussian['n_components']}, covariance_type={best_params_gaussian['covariance_type']})")
plt.colorbar(label='Cluster')
plt.show()

# 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(reduced_data_3d[:, 0], reduced_data_3d[:, 1], reduced_data_3d[:, 2], c=gaussian_with_best_labels, cmap='viridis')
ax.set_title(f"GMM Clustering 3D (n_components={best_params_gaussian['n_components']}, covariance_type={best_params_gaussian['covariance_type']})")
plt.colorbar(scatter, label='Cluster')
plt.show()

In [ ]:
results_df = pd.DataFrame(results_gaussian)
plt.figure(figsize=(15, 5))

plt.subplot(131)
for cov_type in results_df['covariance_type'].unique():
    data = results_df[results_df['covariance_type'] == cov_type]
    plt.plot(data['n_components'], data['silhouette'], label=cov_type, marker='o')
plt.xlabel('n_components')
plt.ylabel('Silhouette Score')
plt.legend()
plt.title('Silhouette Score vs. Parameters')

plt.subplot(132)
for cov_type in results_df['covariance_type'].unique():
    data = results_df[results_df['covariance_type'] == cov_type]
    plt.plot(data['n_components'], data['calinski'], label=cov_type, marker='o')
plt.xlabel('n_components')
plt.ylabel('Calinski-Harabasz Score')
plt.legend()
plt.title('Calinski-Harabasz Score vs. Parameters')

plt.subplot(133)
for cov_type in results_df['covariance_type'].unique():
    data = results_df[results_df['covariance_type'] == cov_type]
    plt.plot(data['n_components'], data['davies'], label=cov_type, marker='o')
plt.xlabel('n_components')
plt.ylabel('Davies-Bouldin Score')
plt.legend()
plt.title('Davies-Bouldin Score vs. Parameters')

plt.tight_layout()
plt.show()

### HDBSCAN

In [ ]:
def apply_hdbscan(X, min_cluster_size, min_samples):
    hdbscan = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples)
    labels = hdbscan.fit_predict(X)
    return labels

In [ ]:
param_grid_hdbscan = {
    'min_cluster_size': range(5, 51, 5),
    'min_samples': range(1, 11)
}

best_score_hdbscan = -np.inf
best_params_hdbscan = None
results_hdbscan = []

for params in ParameterGrid(param_grid_hdbscan):
    labels = apply_hdbscan(df_encoded_all_esc, params['min_cluster_size'], params['min_samples'])
    silhouette, calinski, davies = evaluate_clustering(df_encoded_all_esc, labels)
    
    results_hdbscan.append({
        'min_cluster_size': params['min_cluster_size'],
        'min_samples': params['min_samples'],
        'silhouette': silhouette,
        'calinski': calinski,
        'davies': davies
    })
    
    if silhouette > best_score_hdbscan:
        best_score_hdbscan = silhouette
        best_params_hdbscan = params
    
    print(f"min_cluster_size: {params['min_cluster_size']}, min_samples: {params['min_samples']}, silhouette: {silhouette:.4f}")

print("\nBest parameters:")
print(best_params_hdbscan)
print(f"Best silhouette score: {best_score_hdbscan:.4f}")

Como o código acima demora uma eternidade para executar, aqui está o que estamos buscando:

**Resultados do modelo HDBSCAN**

| Output           | Valor                |
|------------------|----------------------|
| min_cluster_size | 5                    |
| min_samples      | 1                    |
| silhouette       | 0.19251943299001167  |
| calinski         | 39.60968216846142    |
| davies           | 1.2197120808134985   |

Esta tabela apresenta os resultados obtidos com o modelo HDBSCAN, incluindo os hiperparâmetros utilizados (min_cluster_size e min_samples) e as métricas de avaliação (silhouette score, índice Calinski-Harabasz e índice Davies-Bouldin).

In [ ]:
hdbscan_with_best_labels = apply_hdbscan(df_encoded_all_esc,min_cluster_size=5, min_samples=1)

In [ ]:
# 2D
plt.figure(figsize=(10, 8))
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=hdbscan_with_best_labels, cmap='viridis')
plt.title(f"HDBSCAN Clustering 2D (min_cluster_size={best_params_hdbscan['min_cluster_size']}, min_samples={best_params_hdbscan['min_samples']})")
plt.colorbar(label='Cluster')
plt.show()

# 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(reduced_data_3d[:, 0], reduced_data_3d[:, 1], reduced_data_3d[:, 2], c=hdbscan_with_best_labels, cmap='viridis')
ax.set_title(f"HDBSCAN Clustering 3D (min_cluster_size={best_params_hdbscan['min_cluster_size']}, min_samples={best_params_hdbscan['min_samples']})")
plt.colorbar(scatter, label='Cluster')
plt.show()

In [ ]:
results_df = pd.DataFrame(results_hdbscan)
plt.figure(figsize=(15, 15))

plt.subplot(221)
for min_samples in results_df['min_samples'].unique():
    data = results_df[results_df['min_samples'] == min_samples]
    plt.plot(data['min_cluster_size'], data['silhouette'], label=f'min_samples={min_samples}', marker='o')
plt.xlabel('min_cluster_size')
plt.ylabel('Silhouette Score')
plt.legend(title='min_samples', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('Silhouette Score vs. Parameters')

plt.subplot(222)
for min_samples in results_df['min_samples'].unique():
    data = results_df[results_df['min_samples'] == min_samples]
    plt.plot(data['min_cluster_size'], data['calinski'], label=f'min_samples={min_samples}', marker='o')
plt.xlabel('min_cluster_size')
plt.ylabel('Calinski-Harabasz Score')
plt.legend(title='min_samples', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('Calinski-Harabasz Score vs. Parameters')

plt.subplot(223)
for min_samples in results_df['min_samples'].unique():
    data = results_df[results_df['min_samples'] == min_samples]
    plt.plot(data['min_cluster_size'], data['davies'], label=f'min_samples={min_samples}', marker='o')
plt.xlabel('min_cluster_size')
plt.ylabel('Davies-Bouldin Score')
plt.legend(title='min_samples', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('Davies-Bouldin Score vs. Parameters')

plt.subplot(224)
scatter = plt.scatter(results_df['min_cluster_size'], results_df['min_samples'], 
                      c=results_df['silhouette'], cmap='viridis', 
                      s=50, alpha=0.6)
plt.colorbar(scatter, label='Silhouette Score')
plt.xlabel('min_cluster_size')
plt.ylabel('min_samples')
plt.title('HDBSCAN Parameters vs. Silhouette Score')

plt.tight_layout()
plt.show()

### K-means

Para o K-means, também fazer realizar a busca pelos melhores hiperparâmetros.

In [ ]:
param_distributions = {
    'n_clusters': range(2, 21),
    'init': ['k-means++', 'random'],
    'max_iter': [100, 200, 300],
    'n_init': [10, 20, 30]
}

best_score_kmeans = -np.inf
best_params_kmeans = None
results_kmeans = []

n_iter = 100

# Gerando combinações aleatórias de parâmetros
param_list = list(ParameterSampler(param_distributions, n_iter=n_iter, random_state=42))

for params in param_list:
    kmeans = KMeans(**params, random_state=42)
    labels = kmeans.fit_predict(df_encoded_all_esc)
    silhouette, calinski, davies = evaluate_clustering(df_encoded_all_esc, labels)
    
    results_kmeans.append({
        'n_clusters': params['n_clusters'],
        'init': params['init'],
        'max_iter': params['max_iter'],
        'n_init': params['n_init'],
        'silhouette': silhouette,
        'calinski': calinski,
        'davies': davies
    })
    
    if silhouette > best_score_kmeans:
        best_score_kmeans = silhouette
        best_params_kmeans = params
    
    print(f"n_clusters: {params['n_clusters']}, init: {params['init']}, "
          f"max_iter: {params['max_iter']}, n_init: {params['n_init']}, "
          f"silhouette: {silhouette:.4f}")

print("\nBest parameters:")
print(best_params_kmeans)
print(f"Best silhouette score: {best_score_kmeans:.4f}")

Como o código acima demora uma eternidade para executar, aqui está o que estamos buscando:

**Resultados do modelo K-means**

| Output     | Valor                |
|------------|----------------------|
| n_clusters | 2                    |
| init       | k-means++            |
| max_iter   | 300                  |
| n_init     | 30                   |
| silhouette | 0.3772063033450325   |
| calinski   | 8661.77370603141     |
| davies     | 1.204260433186076    |

Esta tabela apresenta os resultados obtidos com o modelo K-means, incluindo os hiperparâmetros utilizados (n_clusters, init, max_iter e n_init) e as métricas de avaliação (silhouette score, índice Calinski-Harabasz e índice Davies-Bouldin).

In [85]:
kmeans_with_best_params = KMeans(random_state=42, n_clusters=2, init='k-means++', max_iter=300, n_init=30)
labels = kmeans_with_best_params.fit_predict(df_encoded_all_esc)

In [ ]:
# 2D plot
plt.figure(figsize=(10, 8))
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=labels, cmap='viridis')
plt.title("KMeans Clustering 2D (n_clusters=2)")
plt.colorbar(label='Cluster')
plt.show()

# 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(reduced_data_3d[:, 0], reduced_data_3d[:, 1], reduced_data_3d[:, 2], c=labels, cmap='viridis')
ax.set_title("KMeans Clustering 3D (n_clusters=2)")
plt.colorbar(scatter, label='Cluster')
plt.show()

In [ ]:
results_df = pd.DataFrame(results_kmeans)

plt.figure(figsize=(15, 5))

plt.subplot(131)
plt.plot(results_df['n_clusters'], results_df['silhouette'], marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs. Number of Clusters')

plt.subplot(132)
plt.plot(results_df['n_clusters'], results_df['calinski'], marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Calinski-Harabasz Score')
plt.title('Calinski-Harabasz Score vs. Number of Clusters')

plt.subplot(133)
plt.plot(results_df['n_clusters'], results_df['davies'], marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Davies-Bouldin Score')
plt.title('Davies-Bouldin Score vs. Number of Clusters')

plt.tight_layout()
plt.show()

## Análise dos Métodos de Seleção de Clusters para o K-means

Realizamos duas abordagens principais para determinar o número ideal de clusters:

1. **Hyperparameter Tuning com Silhouette Score**: Esta análise sugeriu que 2 clusters seriam ideais, baseado na maximização do score de silhueta.

2. **Método do Cotovelo (Elbow Method)**: Este método indicou que 4 clusters seriam mais apropriados.

### Decisão: Utilização de 4 Clusters

Apesar do Silhouette Score favorecer 2 clusters, optamos por utilizar 4 clusters. Esta decisão se baseia nas seguintes considerações, alinhadas com o propósito do projeto:

1. **Granularidade na Segmentação**: Com 4 clusters, podemos obter uma segmentação mais detalhada dos colaboradores, permitindo uma análise mais refinada dos diferentes perfis de saúde e comportamentos de utilização do plano de saúde.

2. **Personalização de Programas de Saúde**: Um dos objetivos principais é "criar programas de saúde que venham de encontro ao perfil e necessidade dos colaboradores". Quatro grupos permitem uma maior customização desses programas, atendendo de forma mais precisa às necessidades específicas de cada segmento.

3. **Alinhamento com a Complexidade do Problema**: Considerando a variedade de fatores que influenciam a saúde dos colaboradores (como doenças crônicas, hábitos de vida, perfil demográfico), 4 clusters oferecem uma representação mais realista dessa complexidade.

4. **Potencial para Insights Mais Ricos**: Com uma segmentação mais detalhada, aumentamos a probabilidade de descobrir padrões e tendências que poderiam passar despercebidos em uma divisão mais ampla.

5. **Flexibilidade na Criação de Programas**: Quatro clusters nos dão mais flexibilidade para "fazer o que faz sentido e não somente por práticas de mercado", como mencionado nos objetivos do projeto.

6. **Suporte à Análise Preditiva**: Para "criar modelos de previsão e tendências", uma segmentação em 4 grupos nos permite capturar nuances mais sutis no comportamento dos colaboradores, potencialmente melhorando a precisão das previsões.

### Conclusão

Embora o Silhouette Score tenha favorecido 2 clusters, a decisão de usar 4 clusters está mais alinhada com os objetivos específicos deste projeto para a Unipar. Esta abordagem nos permite uma análise mais detalhada e personalizada, crucial para o desenvolvimento de programas de saúde eficazes e direcionados. Continuaremos monitorando e validando esta decisão à medida que avançamos no projeto, sempre abertos a ajustes baseados em insights adicionais ou feedback dos stakeholders.

In [ ]:
kmeans = KMeans(n_clusters=4,init='k-means++',random_state=42,max_iter=300, n_init=30)
labels = kmeans.fit_predict(df_encoded_all_esc)

silhouette, calinski, davies = evaluate_clustering(df_encoded_all_esc, labels)
print(silhouette, calinski, davies)

Por causa da nossa decisão de continuar com o agrupamento em 4 clusters, as métricas passam a ser:

**Resultados do modelo K-means**

| Output     | Valor                |
|------------|----------------------|
| n_clusters | 4                    |
| init       | k-means++            |
| max_iter   | 300                  |
| n_init     | 30                   |
| silhouette | 0.22550012925911192  |
| calinski   | 8187.454893885175    |
| davies     | 1.7098871435121716   |

Esta tabela apresenta os resultados obtidos com o modelo K-means, incluindo os hiperparâmetros utilizados (n_clusters, init, max_iter e n_init) e as métricas de avaliação (silhouette score, índice Calinski-Harabasz e índice Davies-Bouldin).

# Comparativas dos modelos

| Modelo           | Silhouette Score | Índice Calinski-Harabasz | Índice Davies-Bouldin |
|------------------|-------------------|--------------------------|------------------------|
| K-means          | 0.2255            | 8187.4549                | 1.7099                 |
| HDBSCAN          | 0.1925            | 39.6097                  | 1.2197                 |
| Gaussian Mixture | 0.2254            | 8185.1803                | 1.7107                 |
| DBSCAN           | 0.1925            | 912.7302                 | 1.4956                 |

## Análise Comparativa

1. **Silhouette Score**:
   - K-means e Gaussian Mixture têm os melhores scores (0.2255 e 0.2254 respectivamente), indicando uma melhor separação e coesão dos clusters.
   - HDBSCAN e DBSCAN têm scores mais baixos (ambos cerca de 0.1925), sugerindo clusters menos bem definidos.

2. **Índice Calinski-Harabasz**:
   - K-means e Gaussian Mixture têm os valores mais altos (8187 e 8185 respectivamente), indicando clusters mais densos e bem separados.
   - DBSCAN tem um valor moderado (912), enquanto HDBSCAN tem um valor muito baixo (39.6), sugerindo que estes modelos baseados em densidade podem estar identificando estruturas de cluster diferentes.

3. **Índice Davies-Bouldin**:
   - HDBSCAN tem o menor valor (1.2197), seguido por DBSCAN (1.4956), indicando uma melhor separação entre clusters para estes modelos baseados em densidade.
   - K-means e Gaussian Mixture têm valores mais altos (1.7099 e 1.7107 respectivamente), sugerindo uma separação menos clara entre os clusters.

### Conclusões:

1. **K-means e Gaussian Mixture** mostram desempenho muito semelhante em todas as métricas. Eles são superiores em termos de Silhouette Score e Calinski-Harabasz, indicando clusters bem definidos e separados. No entanto, seu índice Davies-Bouldin mais alto sugere que pode haver alguma sobreposição entre os clusters.

2. **HDBSCAN e DBSCAN** têm Silhouette Scores mais baixos, mas HDBSCAN se destaca com o melhor (menor) índice Davies-Bouldin. Isso sugere que, embora os clusters possam não ser tão bem separados internamente, eles são mais distintos uns dos outros.

3. O **baixo índice Calinski-Harabasz do HDBSCAN** é notável e pode indicar que este modelo está identificando uma estrutura de cluster muito diferente dos outros modelos, possivelmente capturando clusters de formas não convencionais ou lidando melhor com ruído nos dados.

4. **DBSCAN** parece ter um desempenho intermediário, com um bom equilíbrio entre as diferentes métricas.

###  Modelo MeanShift

O algoritmo Mean Shift é uma técnica de agrupamento (clustering) usado em aprendizado de máquina e visão computacional para identificar regiões densas em um espaço de características. Ele é frequentemente aplicado em problemas de segmentação de imagem e rastreamento de objetos.

In [38]:
df_meanshift = df_cleaned.copy()

#### Pré-processamento e Análise de Componentes Principais (PCA)
Nesta seção, realizamos o pré-processamento dos dados e aplicamos a Análise de Componentes Principais (PCA) para reduzir a dimensionalidade do conjunto de dados.

In [ ]:
# Definindo num_cols e cat_cols
num_cols = df_meanshift.select_dtypes(include=['int32', 'int64', 'float64']).columns.tolist()
cat_cols = df_meanshift.select_dtypes(include=['object']).columns.tolist()

# Pipeline para pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)  # Forçar saída densa
    ]
)

# PCA com 2 componentes
pca = PCA(n_components=2)

# Pipeline para pré-processamento e PCA
pca_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('pca', pca)
])

# Aplicar o pipeline e obter os componentes principais
X_pca = pca_pipeline.fit_transform(df_cleaned)

# Criar o DataFrame para variância explicada
df_var = pd.DataFrame({
    'var': pca.explained_variance_ratio_,
    'PC': ['PC1', 'PC2']
})

# Gráfico 1: PCA scatter plot
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=X_pca[:, 1], cmap='viridis')  # Ajuste no parâmetro 'c' para color mapping

plt.title("PCA - Componentes Principais")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.colorbar(scatter)

# Ajustar layout
plt.tight_layout()

# Mostrar o gráfico
plt.show()

O gráfico de dispersão gerado pelo código, mostra a projeção dos dados originais em dois novos eixos (Componentes Principais 1 e 2). Esses eixos são combinações lineares das variáveis originais e foram escolhidos para capturar a maior parte da variabilidade dos dados. Cada ponto no gráfico representa uma observação ou amostra do conjunto de dados


<br>

--------------------------------------------------------------------

#### Detecção Automática de `Bandwidth` e Aplicação do MeanShift
Neste trecho de código, aplicamos o algoritmo de clustering MeanShift aos dados transformados pelo PCA e visualizamos os clusters gerados, tanto em 2D quanto em 3D, além de analisar a variância explicada pelos componentes principais.

In [ ]:

# Aplicar o MeanShift
meanshift = MeanShift()
cluster_labels = meanshift.fit_predict(X_pca)

# Adicionar os rótulos de cluster ao DataFrame original
df_meanshift['cluster'] = cluster_labels

# Criando os subplots com 2 linhas e 2 colunas
fig, axs = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico 1: Plot dos clusters (PCA 2D)
axs[0, 0].scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, cmap='viridis', alpha=0.5)
axs[0, 0].set_title('Clusters Identificados pelo MeanShift nos Componentes Principais (PCA)')
axs[0, 0].set_xlabel('Componente Principal 1')
axs[0, 0].set_ylabel('Componente Principal 2')
plt.colorbar(axs[0, 0].collections[0], ax=axs[0, 0], label='Cluster Label')
axs[0, 0].grid(True)

# PCA com 3 componentes
pca_3d = PCA(n_components=3)

# Pipeline para pré-processamento e PCA (com 3 componentes)
pca_pipeline_3d = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('pca', pca_3d)
])

# Aplicar o pipeline e obter os componentes principais
X_pca_3D = pca_pipeline_3d.fit_transform(df_meanshift)

# Gráfico 2: PCA 3D plot
ax_3d = fig.add_subplot(223, projection='3d')
scatter_3d = ax_3d.scatter(X_pca_3D[:, 0], X_pca_3D[:, 1], X_pca_3D[:, 2],
                        c=cluster_labels, cmap='viridis')
ax_3d.set_title('PCA com 3 Componentes')
ax_3d.set_xlabel('Componente Principal 1')
ax_3d.set_ylabel('Componente Principal 2')
ax_3d.set_zlabel('Componente Principal 3')
fig.colorbar(scatter_3d, ax=ax_3d, label='Cluster Label')

# Gráfico 3: Variância explicada pelos componentes (PCA com 2 componentes)
df_var = pd.DataFrame({
    'var': pca.explained_variance_ratio_,
    'PC': ['PC1', 'PC2']
})

sns.barplot(x='PC', y="var", data=df_var, color="c", ax=axs[0, 1])
axs[0, 1].set_title("Variância Explicada por Componente (PCA 2D)")
axs[0, 1].set_ylabel("Proporção da Variância Explicada")
axs[0, 1].set_xlabel("Componentes Principais")

# Gráfico 4: Variância explicada pelos componentes (PCA com 3 componentes)
df_var_3D = pd.DataFrame({
    'var': pca_3d.explained_variance_ratio_,
    'PC': ['PC1', 'PC2', 'PC3']
})

sns.barplot(x='PC', y="var", data=df_var_3D, color="c", ax=axs[1, 1])
axs[1, 1].set_title("Variância Explicada por Componente (PCA 3D)")
axs[1, 1].set_ylabel("Proporção da Variância Explicada")
axs[1, 1].set_xlabel("Componentes Principais")

# Ajuste do layout para evitar sobreposição
plt.tight_layout()

# Mostrar o gráfico
plt.show()

##### Gráfico Superior Esquerdo: PCA com 2 Componentes (2D)
Este gráfico 2D exibe a projeção dos dados em dois componentes principais (PC1 e PC2). Novamente, os clusters identificados pelo MeanShift são representados por diferentes cores. O gráfico permite uma visão mais rica das relações entre as amostras ao incluir um terceiro eixo.

##### Gráfico Superior Direito: Variância Explicada (PCA 2D)
Este gráfico de barras mostra a proporção da variância explicada pelos dois primeiros componentes principais. Cada componente (PC1 e PC2) explica cerca de 14% da variância dos dados. Juntos, capturam aproximadamente 28% da variância total.

##### Gráfico Inferior Esquerdo: PCA com 3 Componentes (3D)
Este gráfico 3D exibe a projeção dos dados em três componentes principais (PC1, PC2 e PC3). Novamente, os clusters identificados pelo MeanShift são representados por diferentes cores. O gráfico permite uma visão mais rica das relações entre as amostras ao incluir um terceiro eixo.

##### Gráfico Inferior Direito: Variância Explicada (PCA 3D)
Aqui, o gráfico de barras mostra a proporção da variância explicada pelos três primeiros componentes principais. O PC1 e o PC2 explicam cerca de 14%, enquanto o PC3 explica aproximadamente 10%. Isso indica que esses três componentes capturam cerca de 38% da variância total dos dados.

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### Análise de Clusters Identificados pelo MeanShift
O código a seguir, divide o DataFrame `df_meanshift` em subgrupos com base na coluna `'cluster'`, gerada pelo algoritmo MeanShift. Ele conta o número de elementos em cada cluster e exibe estatísticas descritivas (média, desvio padrão, etc.) para cada um. Também mostra a distribuição de variáveis categóricas selecionadas, permitindo uma análise detalhada de cada cluster.

In [ ]:
df_mean_shift_cluster_0 = df_meanshift[df_meanshift['cluster'] == 0]
df_mean_shift_cluster_1 = df_meanshift[df_meanshift['cluster'] == 1]
df_mean_shift_cluster_2 = df_meanshift[df_meanshift['cluster'] == 2]
df_mean_shift_cluster_3 = df_meanshift[df_meanshift['cluster'] == 3]




# Contagem de elementos em cada cluster
print("Número de elementos em cada grupo:")
print(len(df_mean_shift_cluster_0), 
      len(df_mean_shift_cluster_1), 
      len(df_mean_shift_cluster_2), 
      len(df_mean_shift_cluster_3))


# Estatísticas descritivas dos 4 clusters
for cluster in range(4): 
    print(f"\nCluster {cluster} Statistics:")
    cluster_data = df_meanshift[df_meanshift['cluster'] == cluster]
    

    print(cluster_data.describe())
    
    # Frequências das variáveis categóricas
    for col in ['faixa_etaria_sinistro', 'descricao_plano_sinistro', 'tipo_utilizacao_sinistro', 'descricao_servico_sinistro']:
        print(f"\n{col} distribution in cluster {cluster}:")
        print(cluster_data[col].value_counts())


##### Interpretação do Output

O output fornecido apresenta uma análise detalhada dos clusters formados pelo algoritmo MeanShift. Aqui estão os principais pontos que podem ser interpretados a partir dos dados:

1. **Distribuição dos Elementos nos Clusters**:
   - **Cluster 0**: 21.299 elementos
   - **Cluster 1**: 10.385 elementos
   - **Cluster 2**: 5.107 elementos
   - **Cluster 3**: 2.117 elementos

2. **Cluster 0**:
   - Este cluster é o maior, com 21.299 elementos.
   - A maioria dos segurados está na faixa etária entre 34 e 48 anos.
   - O plano mais comum é o `TQN2`, seguido por `NP2X`.
   - O tipo de utilização predominante é "REDE".
   - Os serviços mais frequentemente utilizados incluem "CONSULTA CONSULTORIO" e "SESSAO DE PSICOTERAPIA INDIVIDUAL".

3. **Cluster 1**:
   - Com 10.385 elementos, é o segundo maior cluster.
   - A faixa etária dominante é também entre 34 e 48 anos, mas há uma maior concentração de segurados de 34 a 38 anos.
   - Novamente, o plano `TQN2` é o mais prevalente.
   - A maioria dos segurados utiliza a "REDE" para serviços, com "CONSULTA CONSULTORIO" sendo o serviço mais comum.

4. **Cluster 2**:
   - Contém 5.107 elementos, com uma clara predominância de segurados com 59 anos ou mais (4.812 elementos).
   - Os planos `TN1E` e `TNQ2` são os mais comuns.
   - Os serviços mais comuns incluem "CONSULTA CONSULTORIO" e exames laboratoriais como "CREATININA" e "UREIA".

5. **Cluster 3**:
   - Este cluster tem 2.117 elementos.
   - Como no Cluster 2, há uma alta concentração de segurados de 59 anos ou mais (1.981 elementos).
   - Os planos mais comuns são `TN1E` e `TQN2`.
   - Serviços médicos utilizados são semelhantes aos do Cluster 2, com uma ênfase em "CONSULTA CONSULTORIO".


##### Conclusões

- **Distribuição Desigual**: Os clusters são bastante desiguais em tamanho, com os dois primeiros clusters (0 e 1) contendo a maioria dos elementos, o que pode indicar uma maior homogeneidade nos dados desses grupos.
- **Faixas Etárias**: Há uma distinção clara de faixas etárias entre os clusters, especialmente entre os segurados mais jovens (clusters 0 e 1) e os mais velhos (clusters 2 e 3).
- **Utilização dos Serviços**: Certos serviços médicos e planos de saúde dominam dentro de cada cluster, o que pode ser útil para segmentação de mercado ou para direcionar campanhas específicas.

-----------------------------------------------

#### Teste de Parâmetros do MeanShift com Avaliação do Silhouette Score
Nesta seção, realizamos uma busca manual pelos melhores hiperparâmetros do algoritmo de clustering MeanShift, utilizando métricas de avaliação para determinar a qualidade dos clusters formados.

In [ ]:
def test_meanshift_clustering(X, bandwidth_values):
    results = []
    
    for bandwidth in bandwidth_values:
        model = MeanShift(bandwidth=bandwidth, bin_seeding=True)
        labels = model.fit_predict(X)
        n_clusters = len(np.unique(labels))
        print(f"Bandwidth: {bandwidth}, Number of Clusters: {n_clusters}")
        if n_clusters > 1:
            score = silhouette_score(X, labels)
            results.append((bandwidth, n_clusters, score))
        else:
            results.append((bandwidth, n_clusters, np.nan))
    
    return results

# Testar diferentes valores de bandwidth
bandwidth_values = [0.1, 0.5, 1, 2, 3, 4]
results = test_meanshift_clustering(X_pca, bandwidth_values)

**Resultados do teste no modelo MeanShift**

| Bandwidth | Number of Clusters |
|-----------|--------------------|
| 0.1       | 560                |
| 0.5       | 8                  |
| 1         | 4                  |
| 2         | 1                  |
| 3         | 1                  |
| 4         |  1                 |


O output acima mostra que o número de clusters formados pelo MeanShift varia significativamente com o valor de bandwidth.

 - Com bandwidth = 0.1, o algoritmo forma 560 clusters, indicando uma fragmentação excessiva dos dados.

 - À medida que o bandwidth aumenta, o número de clusters diminui, sugerindo uma maior coesão dos dados.

 - Para bandwidth ≥ 2, apenas 1 cluster é formado, indicando um excesso de suavização.
  
**Hipótese**: Um valor intermediário de bandwidth (entre 0.5 e 1) parece mais adequado, evitando tanto a fragmentação excessiva quanto a aglomeração dos dados em um único cluster.


#### Teste de Clustering com MeanShift e Diferentes Valores de Bandwidth
Nesta seção, realizamos um teste do algoritmo MeanShift utilizando diferentes valores para o parâmetro `bandwidth`. O objetivo é avaliar como a variação deste parâmetro afeta o número de clusters formados e a qualidade dos _clusters_, medida pelo _Silhouette Score_.

In [ ]:
def custom_scorer(estimator, X):
    labels = estimator.fit_predict(X)
    unique_labels = np.unique(labels)
    
    if len(unique_labels) > 1:
        silhouette = silhouette_score(X, labels)
        davies_bouldin = davies_bouldin_score(X, labels)
        calinski_harabasz = calinski_harabasz_score(X, labels)
        return silhouette, davies_bouldin, calinski_harabasz
    else:
        return float('nan'), float('nan'), float('nan')  # Retorna nan para todas as métricas se houver menos de dois clusters

def grid_search_meanshift(X):
    # Define os parâmetros a serem ajustados
    param_grid = {
        'bandwidth': [0.5, 1],
        'max_iter': [300, 500],
        'min_bin_freq': [1, 2, 3],
        'cluster_all': [True, False]
    }
    
    model = MeanShift(bin_seeding=True)
    
    # Vamos criar listas para armazenar as métricas
    best_silhouette = -1
    best_davies_bouldin = float('inf')
    best_calinski_harabasz = -1
    best_params = None

    for bandwidth in param_grid['bandwidth']:
        for max_iter in param_grid['max_iter']:
            for min_bin_freq in param_grid['min_bin_freq']:
                for cluster_all in param_grid['cluster_all']:
                    model.set_params(bandwidth=bandwidth, max_iter=max_iter, min_bin_freq=min_bin_freq, cluster_all=cluster_all)
                    
                    try:
                        silhouette, davies_bouldin, calinski_harabasz = custom_scorer(model, X)
                        
                        # Aqui decidimos qual conjunto de parâmetros é o "melhor" com base em uma das métricas
                        # ou uma combinação delas. Neste caso, priorizamos silhouette score
                        if silhouette > best_silhouette:
                            best_silhouette = silhouette
                            best_davies_bouldin = davies_bouldin
                            best_calinski_harabasz = calinski_harabasz
                            best_params = {
                                'bandwidth': bandwidth,
                                'max_iter': max_iter,
                                'min_bin_freq': min_bin_freq,
                                'cluster_all': cluster_all
                            }
                    except ValueError as e:
                        print(f"Erro ao calcular as métricas: {e}")
                        continue
    
    return best_params, best_silhouette, best_davies_bouldin, best_calinski_harabasz

# Supondo que X seja o resultado do PCA
best_params, best_silhouette, best_davies_bouldin, best_calinski_harabasz = grid_search_meanshift(X_pca)
print(f"Melhores Parâmetros (GridSearchCV): {best_params}")
print(f"Melhor Silhouette Score: {best_silhouette}")
print(f"Melhor Davies-Bouldin Index: {best_davies_bouldin}")
print(f"Melhor Calinski-Harabasz Index: {best_calinski_harabasz}")


##### Hipótese com Base no Output

O output indica que o melhor conjunto de parâmetros encontrados para o algoritmo MeanShift inclui um `bandwidth` de 1, `max_iter` de 300, `min_bin_freq` de 1 e `cluster_all` definido como `True`. Este modelo obteve as seguintes métricas:

| Output | Valores |
|-----------|--------------------|
| **Silhouette Score**       | 0.6307                |
| **Davies-Bouldin Index**       | 0.4367                  |
| **Calinski-Harabasz Index**         | 64557.34                  |

#### Conclusão: MeanShift como Modelo Preditivo

Embora o MeanShift tenha mostrado resultados decentes com o melhor conjunto de parâmetros (com métricas de qualidade razoáveis), a comparação com outros modelos (como K-means, com um Silhouette Score de 0.2255, Calinski-Harabasz Index de 8187.45, e Davies-Bouldin Index de 1.7099) sugere que o MeanShift pode não ser a melhor opção para o projeto. 

**Razões para considerar o MeanShift inadequado**:

1. **Sensibilidade ao Bandwidth**: MeanShift é altamente sensível ao parâmetro `bandwidth`, e encontrar o valor ideal pode ser difícil e dependente do domínio. Um valor inadequado pode resultar em clusters mal definidos ou excesso de suavização (formação de um único cluster).

2. **Escalabilidade**: MeanShift não escala bem com grandes conjuntos de dados. Se o projeto envolve grandes volumes de dados, a eficiência do algoritmo pode se tornar um problema.

3. **Clusters Esparsos e Pequenos**: Em alguns casos, MeanShift pode gerar clusters muito pequenos ou muito dispersos, o que pode ser inadequado dependendo da aplicação.

4. **Comparação com Outros Modelos**: Quando comparado com outras abordagens, o modelo testado com o MeanShift apresentou métricas (Silhouette, Calinski-Harabasz, Davies-Bouldin) que não foram significativamente superiores, indicando que o esforço para ajustar o MeanShift pode não compensar.

**Conclusão Geral**: Embora o MeanShift tenha sido capaz de formar clusters razoáveis, a complexidade de ajuste dos hiperparâmetros e as limitações em termos de escalabilidade e flexibilidade sugerem que ele pode não ser a melhor opção para o projeto.

## Introdução do Modelo Final (K-Means)

Após uma análise detalhada do algoritmo MeanShift, escolhemos utilizar o K-Means como modelo final devido ao seu desempenho superior em comparação com os demais modelos testados. O K-Means mostrou-se mais eficaz na segmentação dos dados e apresentou resultados mais consistentes para os objetivos da análise. Além disso, para aprofundar a compreensão e facilitar a interpretação dos clusters gerados, diversos novos gráficos foram criados, posibilitando uma melhor visualização detalhada dos dados. Esses gráficos permitem observar com maior clareza a distribuição dos clusters, bem como suas principais características e relações, contribuindo para uma análise mais completa.

In [ ]:
clusters = sorted(df_k_means_esc_all['group'].unique())
homens = []
mulheres = []
totais = []

for cluster in clusters:
    df_cluster = df_k_means_esc_all[df_k_means_esc_all['group'] == cluster]

    homens_count = df_cluster[df_cluster['sexo_sinistro'] == 'M'].shape[0]
    mulheres_count = df_cluster[df_cluster['sexo_sinistro'] == 'F'].shape[0]

    homens.append(homens_count)
    mulheres.append(mulheres_count)
    totais.append(homens_count + mulheres_count)

# Definir a figura com subplots para quatro clusters
fig, axs = plt.subplots(2, 2, figsize=(12, 12))

# Criar lista de cores: verde escuro para homens, cinza para mulheres
cores = ['#3abd3a', '#d4d6d5']  # Verde escuro e cinza

# Para cada cluster, criar um gráfico de pizza
for i, cluster in enumerate(clusters):
    ax = axs[i // 2, i % 2]  # Organizando em 2x2 subplots
    labels = ['Homens', 'Mulheres']
    sizes = [homens[i], mulheres[i]]
    
    # Adicionando o tamanho das labels
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=cores, 
           textprops={'fontsize': 14})  # Tamanho das labels
    
    ax.set_title(f'Distribuição de Sexo no Cluster {cluster}', fontsize=16)  # Título maior

# Ajustar layout para melhor visualização
plt.tight_layout()
plt.show()




### Análise Exploratória dos Clusters: Gênero

A primeira análise exploratória dos clusters focou na variável sexo, com o objetivo de identificar possíveis correlações entre a distribuição de sexo e a segmentação dos clusters. Para isso, foram gerados gráficos de pizza para cada cluster, permitindo uma visualização clara dessa divisão.

#### Resultados por Cluster

#### Cluster 0
- **Distribuição:** 50,6% Homens, 49,4% Mulheres
- **Análise:** Observamos uma distribuição equilibrada, indicando que não há uma tendência significativa de preferência por sexo nesse grupo.

#### Cluster 1
- **Distribuição:** 73,7% Homens, 26,3% Mulheres
- **Análise:** A predominância masculina é marcante, sugerindo uma clara inclinação desse grupo para o sexo masculino.

#### Cluster 2
- **Distribuição:** 60,1% Homens, 39,9% Mulheres
- **Análise:** Apresenta também uma maioria masculina, embora em menor proporção em relação ao Cluster 1.

#### Cluster 3
- **Distribuição:** 66,5% Homens, 33,5% Mulheres
- **Análise:** Segue a mesma tendência dos clusters anteriores, com uma maioria masculina.

### Conclusão

Esses resultados sugerem que, embora alguns clusters tenham uma distribuição de sexo mais equilibrada, há uma tendência geral de predominância masculina, especialmente nos **Clusters 1** e **3**.


In [ ]:
# Calcular custo médio por cluster
custos_medios = []
clusters = sorted(df_k_means_esc_all['group'].unique())
for cluster in clusters:
    df_cluster = df_k_means_esc_all[df_k_means_esc_all['group'] == cluster]
    valor_total = df_cluster['valor_pago_sinistro'].sum()
    quantidade_total = df_cluster['quantidade'].sum()
    custo_medio = valor_total / quantidade_total
    custos_medios.append(custo_medio)
cores = ['#3ABD3A']  # Verde escuro
# Criar o gráfico
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(range(len(clusters)), custos_medios, color= cores)
# Adicionar rótulos e título
ax.set_xlabel('Clusters')
ax.set_ylabel('Custo Médio (R$)')
ax.set_title('Custo Médio por Utilização em Cada Cluster')
ax.set_xticks(range(len(clusters)))
ax.set_xticklabels([f'Cluster {cluster}' for cluster in clusters])
# Adicionar os valores em cima das barras
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'R$ {height:.2f}',
            ha='center', va='bottom')
plt.tight_layout()
plt.show()

### Análise de Custo Médio por Utilização em Cada Cluster
Nesta análise, buscamos entender a variação no custo médio por utilização entre os diferentes clusters. A visualização gerada por meio de um gráfico de barras permite uma análise clara da disparidade entre os clusters.
### Resultados por Cluster
#### Cluster 0
- **Custo Médio:** R$ 118,09
- **Análise:** O Cluster 0 apresenta um custo médio relativamente equilibrado em comparação com os demais clusters.
#### Cluster 1
- **Custo Médio:** R$ 109,41
- **Análise:** O Cluster 1 mantém uma média similar ao Cluster 0, com um custo de utilização um pouco menor, sugerindo pouca variação entre os dois clusters.
#### Cluster 2
- **Custo Médio:** R$ 95,55
- **Análise:** O Cluster 2 tem o custo médio mais baixo entre todos, indicando uma menor utilização ou serviços menos dispendiosos em comparação com os outros clusters.
#### Cluster 3
- **Custo Médio:** R$ 252,26
- **Análise:** O Cluster 3 se destaca significativamente dos demais, apresentando o maior custo médio, mais que o dobro do valor do Cluster 0. Isso sugere que as utilizações associadas a esse grupo são consideravelmente mais caras.
### Conclusão
A análise sugere que, enquanto os Clusters 0, 1 e 2 possuem custos médios relativamente próximos e equilibrados, o Cluster 3 tem um custo médio de utilização substancialmente mais alto, indicando um comportamento distinto e provavelmente mais caro para os serviços ou produtos associados a esse grupo.

Agora vamos criar quatro gráficos que examinam a distribuição dos planos em cada cluster individualmente.

In [ ]:
clusters = [0, 1, 2, 3]
planCounts = {}

# Obter os nomes únicos dos planos
unique_plans = df_k_means_esc_all['descricao_plano_sinistro'].unique()

# Inicializar listas vazias para cada plano em planCounts
for plan in unique_plans:
    planCounts[plan] = []

# Contar as ocorrências de cada plano por cluster
for cluster in clusters:
    df_cluster = df_k_means_esc_all[df_k_means_esc_all['group'] == cluster]
    counts = df_cluster['descricao_plano_sinistro'].value_counts()    
    for plan in unique_plans:
        planCounts[plan].append(counts.get(plan, 0))

# Definir um mapeamento de cores para cada plano para garantir cores consistentes entre os gráficos
plan_colors = {plan: f'C{i}' for i, plan in enumerate(unique_plans)}

# Criar um gráfico separado para cada cluster, filtrando os planos com 0 registros
for cluster in clusters:
    # Filtrar os planos com 0 registros para este cluster específico
    plans_in_cluster = [plan for plan in unique_plans if planCounts[plan][cluster] > 0]
    
    # Pular clusters sem planos válidos
    if len(plans_in_cluster) == 0:
        continue
    
    # Criar uma lista de tuplas (plano, contagem) para o cluster e ordenar do maior para o menor
    plans_in_cluster_sorted = sorted(
        [(plan, planCounts[plan][cluster]) for plan in plans_in_cluster], 
        key=lambda x: x[1], 
        reverse=False
    )
    
    total_count = sum(plan[1] for plan in plans_in_cluster_sorted)
    
    # Obter a contagem máxima para este cluster para definir o limite dinâmico
    max_count = max(plan[1] for plan in plans_in_cluster_sorted)
    
    # Definir o limite com um buffer de 1000 unidades
    x_limit = max_count + 1000
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plotar as barras horizontais com cores consistentes
    for i, (plan, count) in enumerate(plans_in_cluster_sorted):
        percentage = (count / total_count) * 100 if total_count > 0 else 0
        
        # Criar as barras horizontais usando a cor do mapeamento de cores pré-definido
        bars = ax.barh(plan, count, color=plan_colors[plan], label=plan, height=0.5)
        
        # Adicionar a quantidade e a porcentagem em cada barra
        for bar in bars:
            width = bar.get_width()
            
            # Exibir a quantidade e a porcentagem em cada barra
            ax.annotate(f'{int(width)}',  # Exibir a quantidade
                        xy=(width, bar.get_y() + bar.get_height() / 2),  # Posição do texto
                        xytext=(2, 5),  # Deslocar o texto levemente
                        textcoords="offset points",
                        ha='left', va='center', fontsize=9, weight='bold')  # Ajustar o tamanho da fonte
            
            ax.annotate(f'({percentage:.1f}%)',  # Exibir a porcentagem
                        xy=(width, bar.get_y() + bar.get_height() / 2),  # Posição do texto
                        xytext=(2, -5),  # Deslocar o texto da porcentagem levemente
                        textcoords="offset points",
                        ha='left', va='center', fontsize=8)  # Ajustar o tamanho da fonte

    # Definir os títulos e rótulos do gráfico
    ax.set_title(f'Distribuição de Planos no Cluster {cluster} (Quantidade e Porcentagem)')
    ax.set_xlabel('Quantidade de Planos')
    ax.set_ylabel('Planos')
    
    # Definir um limite específico no eixo x com base no valor máximo e um offset de 1000
    ax.set_xlim(0, x_limit)
    
    # Exibir a legenda
    ax.legend(title='Plano', loc='lower right')

    # Ajustar o layout
    plt.tight_layout()
    plt.show()


### Análise da Distribuição de Planos por Cluster

#### Cluster 0
No Cluster 0, o plano **TN1E** é o mais predominante, com **42.4%** dos planos, totalizando **2984** unidades. Logo em seguida, o plano **TNQ2** aparece com **25.7%** (1811 planos), e o **TQN2** representa **24.6%** (1733 planos). Já os planos **NP6X**, **NP2X** e **TP8X** possuem uma presença menor, com **3.0%** (209), **2.3%** (163) e **2.0%** (138) respectivamente. Este cluster é fortemente dominado pelos três primeiros planos, que juntos representam mais de 90% dos planos alocados.

#### Cluster 1
No Cluster 1, o plano **TQN2** é altamente dominante, correspondendo a **83.0%** dos planos, com **12,228** planos no total. O segundo plano mais relevante é o **NP2X**, com **9.4%** (1392 planos), seguido pelo **NP6X** com **4.1%** (597). O plano **TNE1** aparece em uma quantidade reduzida, com **3.4%** (505). Os demais planos, como **TP8X**, **TN1E** e **QN06**, têm uma presença praticamente insignificante, representando menos de **1%** da distribuição de planos neste cluster.

#### Cluster 2
No Cluster 2, o plano **TQN2** novamente é o mais prevalente, com **67.4%** dos planos, totalizando **9072**. O plano **NP2X** aparece como o segundo mais comum, com **17.3%** (2324), seguido pelo **TNE1** com **7.5%** (1005 planos). O plano **NP6X** ocupa **5.5%** (738 planos) e o **TP8X** apenas **2.3%** (314 planos). Planos como **QN06** e **TNQ2** são basicamente inexistentes nesse cluster, representando **0.1%** e **0.0%** respectivamente.

#### Cluster 3
No Cluster 3, o plano **TQN2** continua liderando com **67.4%**, somando **2476** planos. Em segundo lugar aparece o **NP2X** com **17.9%** (656), seguido pelo **NP6X** com **9.0%** (330). O plano **TP8X** tem uma pequena presença de **4.0%** (148). Os planos **TN1E**, **TNQ2** e **TNE1** têm presenças não muito significativas, com **0.6%** e **0.4%**, correspondendo a valores muito pequenos (entre 22 e 16 planos).

### Conclusão dos planos
Em todos os clusters, o plano **TQN2** se destaca como o mais predominante de todos, especialmente no Cluster 1, onde representa a maioria por **83.0%**. Os planos **NP2X** e **NP6X** também aparecem de forma consistente nos clusters, com variações de presença, sendo mais relevantes nos Clusters 1, 2 e 3. Por outro lado, planos como **TP8X**, **TN1E**, **TNQ2** e **QN06** aparecem em quantidades muito pequenas, sem relevância significativa em nenhum dos clusters. Essa distribuição sugere que o **TQN2** é o plano mais utilizado de forma consistente, enquanto os outros planos têm presenças muito mais localizadas e menores.


Para entender melhor as diferenças, iremos criar gráficos de pizza que nos permitirão visualizar a distribuição das empresas em cada cluster.

In [ ]:
import matplotlib.pyplot as plt

clusters = [0, 1, 2, 3]
companyCounts = {}

# Obter os nomes únicos das empresas
unique_companies = df_k_means_esc_all['nome_empresa_sinistro'].unique()

# Inicializar listas vazias para cada empresa em companyCounts
for company in unique_companies:
    companyCounts[company] = []

# Contar as ocorrências de cada empresa por cluster
for cluster in clusters:
    df_cluster = df_k_means_esc_all[df_k_means_esc_all['group'] == cluster]
    counts = df_cluster['nome_empresa_sinistro'].value_counts()
    for company in unique_companies:
        companyCounts[company].append(counts.get(company, 0))

# Criar uma figura com 4 subplots (2x2 layout)
fig, axs = plt.subplots(2, 2, figsize=(14, 10))

# Definir as cores para cada empresa (Verde escuro e cinza)
colors = ['#3ABD3A', '#D4D6D5']
company_colors = {company: colors[i % len(colors)] for i, company in enumerate(unique_companies)}

# Inicializar uma lista de legendas (labels) para a tabela
all_labels = set()

# Criar um gráfico de pizza para cada cluster
for i, cluster in enumerate(clusters):
    # Filtrar as empresas com 0 registros para este cluster específico
    companies_in_cluster = [company for company in unique_companies if companyCounts[company][cluster] > 0]

    # Pular clusters sem empresas válidas
    if len(companies_in_cluster) == 0:
        continue

    # Criar uma lista de tuplas (empresa, contagem) para o cluster e ordenar do maior para o menor
    companies_in_cluster_sorted = sorted(
        [(company, companyCounts[company][cluster]) for company in companies_in_cluster],
        key=lambda x: x[1],
        reverse=True
    )

    # Extrair os nomes das empresas e as contagens
    labels = [company for company, count in companies_in_cluster_sorted]
    sizes = [count for company, count in companies_in_cluster_sorted]

    # Atualizar a lista de legendas (para garantir que as cores permaneçam consistentes)
    all_labels.update(labels)

    # Determinar o subplot a ser usado
    ax = axs[i // 2, i % 2]

    # Plotar o gráfico de pizza sem bordas
    wedges, texts, autotexts = ax.pie(sizes, labels=None, colors=[company_colors[company] for company in labels],
                                      autopct='%1.0f%%', startangle=90)

    # Aumentar o tamanho da fonte das porcentagens dentro das pizzas
    for autotext in autotexts:
        autotext.set_fontsize(14)  # Aqui definimos o tamanho da fonte para 14

    # Definir o título do gráfico e ajustar a distância com pad
    ax.set_title(f'Cluster {cluster}', pad=0)

# Ajustar o espaçamento entre os subplots para remover o espaço branco
plt.subplots_adjust(wspace=0, hspace=0)  # Reduzir o espaçamento entre as pizzas

# Adicionar uma legenda global fora da área dos gráficos
fig.legend(wedges, all_labels, title="Empresas", loc="center right", bbox_to_anchor=(1.1, 0.5),
           fontsize=12, labelspacing=1.5)  # Aumentar o espaçamento e o tamanho da fonte da legenda

# Ajustar o layout para garantir que todos os elementos sejam exibidos corretamente
plt.tight_layout()
plt.show()

### Análise da Distribuição de Empresas por Cluster

A **UNIPAR CARBOCLORO S.A.** tem presença dominante nos Clusters 0, 2 e 3, com participações de **98%, 100% e 72%**, respectivamente. Já a **UNIPAR INDUPA DO BRASIL S.A.** possui uma presença pequena de **2% no Cluster 0**, mas é totalmente dominante no **Cluster 1, com 100%**. No **Cluster 3**, a presença é mais equilibrada, com a **UNIPAR INDUPA** detendo **28%**.

## Conclusão da análise final dos Clusters

A base da nossa análise final podemos perceber as principais diferenças entre os Clusters, aqui estão os principais fatores de cada um listado.

#### Cluster 0: 
O Cluster 0 é o terçeiro menor Cluster com em torno de 7 mil segurados, possui um custo de media de R$118.09 sendo a segunda maior media com o custo total de R$874.933,56, é formado apenas por segurados de 54 a 59 anos, 98% sendo da Unipar Indupa e apenas 2% sendo da Carbocloro, possui uma quase perfeita divisão entre sexos, 50.6% sendo Masculino e 49.4% sendo Feminino, o Cluster 0 também é o unico que possui os Planos TN1E e TNQ2 sendo o TN1E o mais prevalente com quase 3 mil planos.

#### Cluster 1: 
O Cluster 1 é o maior Cluster, possuindo perto de 15 mil segurados, possui um custo de média de R$109.41 com o custo total de R$1.777.017,25 sendo o maior custo total entre os Clusters, abrange idades de 18 a 59 anos e é formado por 73.7% por segurados masculino e 26.3% de segurados femininos, possui como seu plano mais utilizado o TQN2, esse Cluster é completamente formado apenas por segurados da Unipar Carbocloro.

#### Cluster 2: 
O Cluster 2 é extremamente semelhante ao Cluster 1, possui em torno de 13 mil segurados tendo sua principal diferença que ao contrario do Cluster 1 que é formado apenas for segurados da Unipar Carbocloro, esse Cluster é completamente formado por apenas segurados da Unipar Indupa, é o segundo maior Cluster, possui um custo de media de R$95.55 com seu custo total de R$1.373.039,73 sendo o segundo maior custo apos o Cluster 1, igual ao Cluster 1 abrange idades de 18 a 59 anos e seu plano mais utilizado é o TQN2, porem possui um aumento na quantidades de segurados de sexo feminino com 39.9% sendo sexo feminino e 60.1% de sexo masculino.

#### Cluster 3: 
O Cluster 3 se diferencia completamente dos outros, sendo apenas formado por tipo de Sinistro de Reembolso enquanto todos os outros são de Rede, possui a menor quantidade de Segurados quase chegando a 4 mil segurados, porem possui o terceiro maior custo de R$1.263.330,72 quase ultrapasando o Cluster 2 com R$1.373.039,73 que possui 13 mil segurados, o custo medio é de R$252.26 sendo o maior custo medio entre todos os clusters, igual ao Clusters 1 e 2 seu plano mais utilizado é o TQN2, 66.5% de seus segurados são do sexo masculino e 33.5% do sexo feminino, é formado por principalmente a Unipar Indupa com 72% de seus segurados sendo de la enqunto 28% são da Carbocloro.